In [ ]:
# 1. Uninstall potential conflicts first
!pip uninstall -y numpy transformers accelerate diffusers

# 2. Install the "Safe Stack"
# numpy<2: Fixes binary incompatibility with PyTorch
# transformers>=4.41: Fixes the 'AutoImageProcessor' import error
# diffusers==0.30.0: Stable version for ControlNet
!pip install "numpy<2.0" "transformers>=4.41.0" "diffusers==0.30.0" "accelerate>=0.30.0" opencv-python

print("Installation Complete. You may proceed to Cell 2.")

In [ ]:
import os
import cv2
import numpy as np
import torch
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from diffusers import (
    ControlNetModel,
    DDPMScheduler,
    AutoencoderKL,
    UNet2DConditionModel,
)
from transformers import AutoTokenizer, CLIPTextModel
from accelerate import Accelerator
from tqdm.auto import tqdm

print("Imports successful! You are ready to train.")

In [ ]:
import os
import cv2
import numpy as np
import torch
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from diffusers import (
    ControlNetModel,
    DDPMScheduler,
    AutoencoderKL,
    UNet2DConditionModel,
)
from transformers import AutoTokenizer, CLIPTextModel
from accelerate import Accelerator
from tqdm.auto import tqdm

# --- CONFIGURATION ---
# Your specific path from the previous messages
DATA_PATH = "/kaggle/input/processed-images/content/drive/MyDrive/ControlNet_Project/data/processed"
MODEL_ID = "runwayml/stable-diffusion-v1-5"
OUTPUT_DIR = "/kaggle/working/controlnet_model"
FIXED_PROMPT = "high quality detailed image"
BATCH_SIZE = 4       # Lower to 1 if you hit Out Of Memory errors
NUM_EPOCHS = 1       # Start with 1 to test
LEARNING_RATE = 1e-5
IMAGE_SIZE = 512

# --- 1. DATASET CLASS ---
class ControlNetDataset(Dataset):
    def __init__(self, data_root, tokenizer, size=512):
        self.data_root = data_root
        self.tokenizer = tokenizer
        self.size = size
        # Find all images recursively or in the top folder
        self.image_paths = []
        for root, dirs, files in os.walk(data_root):
            for file in files:
                if file.lower().endswith(('.png', '.jpg', '.jpeg')):
                    self.image_paths.append(os.path.join(root, file))
        
        print(f"Found {len(self.image_paths)} images.")

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        # Load Image
        img_path = self.image_paths[idx]
        image = cv2.imread(img_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        # Generate Canny Edge (Condition)
        canny_image = cv2.Canny(image, 100, 200)
        canny_image = np.stack([canny_image, canny_image, canny_image], axis=2)

        # Resize
        image = cv2.resize(image, (self.size, self.size))
        canny_image = cv2.resize(canny_image, (self.size, self.size))

        # Normalize Target [-1, 1]
        target_tensor = (image.astype(np.float32) / 127.5) - 1.0
        target_tensor = torch.from_numpy(target_tensor).permute(2, 0, 1)

        # Normalize Condition [0, 1]
        control_tensor = canny_image.astype(np.float32) / 255.0
        control_tensor = torch.from_numpy(control_tensor).permute(2, 0, 1)

        # Tokenize Prompt
        inputs = self.tokenizer(
            FIXED_PROMPT, 
            max_length=self.tokenizer.model_max_length, 
            padding="max_length", 
            truncation=True, 
            return_tensors="pt"
        )

        return {
            "pixel_values": target_tensor,
            "conditioning_pixel_values": control_tensor,
            "input_ids": inputs.input_ids[0]
        }

# --- 2. SETUP ACCELERATOR & MODELS ---
accelerator = Accelerator(mixed_precision="fp16") # Uses Kaggle P100/T4 features

# Load components
tokenizer = AutoTokenizer.from_pretrained(MODEL_ID, subfolder="tokenizer")
noise_scheduler = DDPMScheduler.from_pretrained(MODEL_ID, subfolder="scheduler")
text_encoder = CLIPTextModel.from_pretrained(MODEL_ID, subfolder="text_encoder")
vae = AutoencoderKL.from_pretrained(MODEL_ID, subfolder="vae")
unet = UNet2DConditionModel.from_pretrained(MODEL_ID, subfolder="unet")

# Initialize ControlNet from UNet weights
print("Initializing ControlNet weights from UNet...")
controlnet = ControlNetModel.from_unet(unet)

# Freeze unrelated models
vae.requires_grad_(False)
unet.requires_grad_(False)
text_encoder.requires_grad_(False)

# Set ControlNet to Train
controlnet.train()
controlnet.enable_gradient_checkpointing() # Saves VRAM

# Optimizer
optimizer = torch.optim.AdamW(controlnet.parameters(), lr=LEARNING_RATE)

# Data Loader
dataset = ControlNetDataset(DATA_PATH, tokenizer, size=IMAGE_SIZE)
# Old line
train_dataloader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=0)

# Prepare everything with Accelerator
controlnet, optimizer, train_dataloader = accelerator.prepare(
    controlnet, optimizer, train_dataloader
)

# Move frozen models to GPU
vae.to(accelerator.device, dtype=torch.float16)
unet.to(accelerator.device, dtype=torch.float16)
text_encoder.to(accelerator.device, dtype=torch.float16)


In [ ]:
import sys
import gc
import torch
import torch.nn.functional as F
from torch.cuda.amp import GradScaler, autocast

# --- CONFIGURATION ---
BATCH_SIZE = 1 
GRADIENT_ACCUMULATION_STEPS = 4 
LEARNING_RATE = 1e-5
NUM_EPOCHS = 1
FIXED_PROMPT = "high quality detailed image"

# 1. CLEANUP
torch.cuda.empty_cache()
gc.collect()

if not torch.cuda.is_available():
    sys.exit("🔴 ERROR: GPU not detected.")

device = torch.device("cuda")
print(f"✅ Training on Device: {device}")

# 2. LOAD MODELS
print("Loading models...")
vae = AutoencoderKL.from_pretrained(MODEL_ID, subfolder="vae").to(device)
unet = UNet2DConditionModel.from_pretrained(MODEL_ID, subfolder="unet").to(device)
text_encoder = CLIPTextModel.from_pretrained(MODEL_ID, subfolder="text_encoder").to(device)
tokenizer = AutoTokenizer.from_pretrained(MODEL_ID, subfolder="tokenizer")

# 3. PRE-COMPUTE TEXT EMBEDDINGS (The Memory Hack)
# Since the prompt is always the same, we encode it once and delete the model.
print("Pre-computing text embeddings...")
with torch.no_grad():
    tokens = tokenizer(
        FIXED_PROMPT, 
        max_length=tokenizer.model_max_length, 
        padding="max_length", 
        truncation=True, 
        return_tensors="pt"
    ).input_ids.to(device)
    
    # Save this tensor!
    FIXED_ENCODER_HIDDEN_STATES = text_encoder(tokens)[0].detach()

# DELETE TEXT ENCODER TO SAVE 1GB VRAM
del text_encoder
del tokenizer
torch.cuda.empty_cache()
print("✅ Text Encoder deleted. VRAM freed.")

# 4. SETUP CONTROLNET
controlnet = ControlNetModel.from_unet(unet).to(device)
vae.requires_grad_(False)
unet.requires_grad_(False)

controlnet.train()
controlnet.enable_gradient_checkpointing()

# 5. OPTIMIZER
optimizer = torch.optim.AdamW(controlnet.parameters(), lr=LEARNING_RATE)
scaler = GradScaler()

# 6. DATA LOADER (Batch Size 1)
# Note: We don't need input_ids from the dataset anymore
train_dataloader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=0)

# 7. TRAINING LOOP
print(f"Starting training for {NUM_EPOCHS} epoch(s)...")

for epoch in range(NUM_EPOCHS):
    progress_bar = tqdm(total=len(train_dataloader))
    progress_bar.set_description(f"Epoch {epoch}")
    
    for step, batch in enumerate(train_dataloader):
        
        # A. Move Data
        pixel_values = batch["pixel_values"].to(device)
        control_values = batch["conditioning_pixel_values"].to(device)
        # We ignore batch['input_ids'] because we use the cached one

        # B. AUTOMATIC MIXED PRECISION
        with autocast(dtype=torch.float16):
            # Encode (VAE)
            latents = vae.encode(pixel_values).latent_dist.sample()
            latents = latents * vae.config.scaling_factor

            # Noise
            noise = torch.randn_like(latents)
            timesteps = torch.randint(0, noise_scheduler.config.num_train_timesteps, (latents.shape[0],), device=device).long()
            noisy_latents = noise_scheduler.add_noise(latents, noise, timesteps)

            # ControlNet Forward (Using CACHED embeddings)
            # We expand the cached embedding to match the batch size (even if it's 1)
            batch_hidden_states = FIXED_ENCODER_HIDDEN_STATES.expand(latents.shape[0], -1, -1)

            down_block_res_samples, mid_block_res_sample = controlnet(
                noisy_latents,
                timesteps,
                encoder_hidden_states=batch_hidden_states,
                controlnet_cond=control_values,
                return_dict=False,
            )

            # UNet Forward
            noise_pred = unet(
                noisy_latents,
                timesteps,
                encoder_hidden_states=batch_hidden_states,
                down_block_additional_residuals=down_block_res_samples,
                mid_block_additional_residual=mid_block_res_sample,
            ).sample

            # Loss
            loss = F.mse_loss(noise_pred.float(), noise.float(), reduction="mean")
            loss = loss / GRADIENT_ACCUMULATION_STEPS

        # C. BACKWARD PASS
        scaler.scale(loss).backward()

        if (step + 1) % GRADIENT_ACCUMULATION_STEPS == 0:
            scaler.step(optimizer)
            scaler.update()
            # set_to_none=True saves a tiny bit more memory
            optimizer.zero_grad(set_to_none=True)
            
        progress_bar.update(1)
        progress_bar.set_postfix(loss=loss.detach().item() * GRADIENT_ACCUMULATION_STEPS)

    # Save
    print(f"Saving Epoch {epoch}...")
    save_folder = os.path.join(OUTPUT_DIR, f"checkpoint-epoch-{epoch}")
    controlnet.save_pretrained(save_folder)
    print(f"Saved to {save_folder}")

print("Training Finished!")

In [ ]:
import torch
print(f"Is CUDA available? {torch.cuda.is_available()}")
print(f"Current Device Name: {torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'None'}")

In [ ]:
import torch
import cv2
import os
import random
import numpy as np
from PIL import Image
from diffusers import StableDiffusionControlNetPipeline, ControlNetModel, UniPCMultistepScheduler
import matplotlib.pyplot as plt

# --- 1. CLEANUP & SETUP ---
torch.cuda.empty_cache()
device = "cuda"
base_model_id = "runwayml/stable-diffusion-v1-5"
trained_controlnet_path = "/kaggle/working/controlnet_model/checkpoint-epoch-0"
data_path = "/kaggle/input/processed-images/content/drive/MyDrive/ControlNet_Project/data/processed"

print("🔄 Reloading models for inference...")

# --- 2. LOAD MODELS ---
# Load your trained ControlNet
controlnet = ControlNetModel.from_pretrained(trained_controlnet_path, torch_dtype=torch.float16)

# Load the full pipeline (Text Encoder comes back here)
pipe = StableDiffusionControlNetPipeline.from_pretrained(
    base_model_id,
    controlnet=controlnet,
    torch_dtype=torch.float16,
    safety_checker=None  # Disable safety checker to save RAM/Time
)

# Use a memory-efficient scheduler
pipe.scheduler = UniPCMultistepScheduler.from_config(pipe.scheduler.config)
pipe.enable_model_cpu_offload() # Crucial for Kaggle memory management
print("✅ Models loaded!")

# --- 3. PREPARE INPUT IMAGE ---
# Pick a random image from your dataset to use as a "structure guide"
all_images = []
for root, dirs, files in os.walk(data_path):
    for file in files:
        if file.lower().endswith(('.png', '.jpg', '.jpeg')):
            all_images.append(os.path.join(root, file))

random_image_path = random.choice(all_images)
original_image = cv2.imread(random_image_path)
original_image = cv2.cvtColor(original_image, cv2.COLOR_BGR2RGB)

# Create Canny Edge Map (The Condition)
image = np.array(original_image)
low_threshold = 100
high_threshold = 200
image = cv2.Canny(image, low_threshold, high_threshold)
image = image[:, :, None]
image = np.concatenate([image, image, image], axis=2)
canny_image = Image.fromarray(image)

# --- 4. GENERATE ---
print("🎨 Generating...")
prompt = "high quality detailed image, bright red sports car, matte finish, 8k, realistic"
negative_prompt = "low quality, bad quality, blurry, distorted, messy"

output = pipe(
    prompt,
    negative_prompt=negative_prompt,
    image=canny_image,
    num_inference_steps=20,
    guidance_scale=7.5
).images[0]

# 1. Setup the plot grid again
fig, axs = plt.subplots(1, 3, figsize=(15, 5))

# 2. Plot Original Image (Ground Truth)
# Ensure 'original_image' is the variable holding your loaded car photo
axs[0].imshow(original_image)
axs[0].set_title("Original Image")
axs[0].axis("off")

# 3. Plot Canny Edge (The Outline)
# Ensure 'canny_image' is the variable holding the black & white edges
axs[1].imshow(canny_image)
axs[1].set_title("Canny Edge Map")
axs[1].axis("off")

# 4. Plot Generated Result
# Ensure 'output' is the variable holding the result from the pipe() call
axs[2].imshow(output)
axs[2].set_title("Generated Car")
axs[2].axis("off")

# 5. Show the final plot
plt.show()

In [ ]:
import matplotlib.pyplot as plt

# 1. Define your prompt list
test_prompts = [
    "high quality detailed image, bright red sports car, matte finish",
    "high quality detailed image, midnight blue car, metallic paint",
    "high quality detailed image, car driving on a rainy street at night, neon lights",
    "high quality detailed image, pencil sketch of a car, rough charcoal lines",
]

# 2. Setup Plot
fig, axs = plt.subplots(1, len(test_prompts) + 1, figsize=(20, 5))

# Plot the Canny Edge (The Reference)
axs[0].imshow(canny_image)
axs[0].set_title("Input Structure (Edges)")
axs[0].axis("off")

# 3. Generate and Plot variations
print("🎨 Generating variations...")
for i, prompt in enumerate(test_prompts):
    output = pipe(
        prompt,
        negative_prompt="low quality, bad quality, blurry, distorted, messy",
        image=canny_image,
        num_inference_steps=20,
        guidance_scale=7.5
    ).images[0]
    
    axs[i+1].imshow(output)
    axs[i+1].set_title(f"Prompt: {prompt.split(',')[1][:20]}...") # Show short title
    axs[i+1].axis("off")

plt.show()

In [ ]:
import ipywidgets as widgets
from IPython.display import display
import io
from PIL import Image

# Create the upload widget
uploader = widgets.FileUpload(
    accept='.jpg, .png, .jpeg',  # Accept only images
    multiple=False  # Single file only
)

print("👇 Click below to upload your Reference Photo:")
display(uploader)

In [ ]:
import cv2
import numpy as np
from sklearn.cluster import KMeans
from PIL import Image, ImageDraw
import matplotlib.pyplot as plt

# --- 1. COLOR MAPPING HELPER ---
class ColorMapper:
    def __init__(self):
        # A manual dictionary of common car colors for robust matching
        self.colors = {
            "black": (0, 0, 0), "white": (255, 255, 255), "red": (255, 0, 0),
            "green": (0, 128, 0), "blue": (0, 0, 255), "yellow": (255, 255, 0),
            "cyan": (0, 255, 255), "magenta": (255, 0, 255), "silver": (192, 192, 192),
            "gray": (128, 128, 128), "maroon": (128, 0, 0), "olive": (128, 128, 0),
            "purple": (128, 0, 128), "teal": (0, 128, 128), "navy": (0, 0, 128),
            "orange": (255, 165, 0), "gold": (255, 215, 0), "beige": (245, 245, 220),
            "brown": (165, 42, 42), "lime": (0, 255, 0), "turquoise": (64, 224, 208)
        }

    def get_color_name(self, rgb_tuple):
        # Find the color in dictionary with smallest Euclidean distance
        min_dist = float("inf")
        closest_name = "gray"
        r, g, b = rgb_tuple
        
        for name, value in self.colors.items():
            cr, cg, cb = value
            dist = np.sqrt((r - cr)**2 + (g - cg)**2 + (b - cb)**2)
            if dist < min_dist:
                min_dist = dist
                closest_name = name
        return closest_name

def get_dominant_colors(image_cv2, k=4):
    """Extracts k dominant colors and returns their RGB values and Names."""
    # Resize to speed up KMeans
    img_small = cv2.resize(image_cv2, (64, 64))
    img_small = cv2.cvtColor(img_small, cv2.COLOR_BGR2RGB)
    pixels = img_small.reshape(-1, 3)
    
    # Clustering
    kmeans = KMeans(n_clusters=k, n_init=10)
    kmeans.fit(pixels)
    colors = kmeans.cluster_centers_.astype(int)
    
    # Map to names
    mapper = ColorMapper()
    color_names = [mapper.get_color_name(tuple(c)) for c in colors]
    
    return colors, color_names



In [ ]:
import io
import cv2
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image

# --- 1. LOAD UPLOADED IMAGE ---
if not uploader.value:
    print("⚠️ Please upload an image in the previous cell first!")
else:
    try:
        # ATTEMPT 1: New ipywidgets (Version 8+) - Returns a tuple of dicts
        if isinstance(uploader.value, tuple):
            uploaded_file = uploader.value[0]
            content = uploaded_file['content']
            uploaded_filename = uploaded_file.get('name', 'uploaded_image')
        # ATTEMPT 2: Old ipywidgets (Version 7) - Returns a dict of dicts
        else:
            uploaded_filename = list(uploader.value.keys())[0]
            content = uploader.value[uploaded_filename]['content']

        # Convert bytes to OpenCV format
        image_stream = io.BytesIO(content)
        pil_image = Image.open(image_stream).convert("RGB")
        original_image = np.array(pil_image)
        
        # Convert RGB to BGR for OpenCV functions
        original_image_cv2 = cv2.cvtColor(original_image, cv2.COLOR_RGB2BGR)

        print(f"✅ Successfully loaded: {uploaded_filename}")

        # --- 2. EXTRACT COLORS ---
        # (Re-using the get_dominant_colors function we defined earlier)
        palette_rgb, palette_names = get_dominant_colors(original_image_cv2, k=4)
        palette_str = ", ".join(set(palette_names))
        print(f"🎨 Detected Palette: {palette_str}")

        # --- 3. PREPARE CANNY EDGES ---
        canny_edges = cv2.Canny(original_image, 100, 200)
        canny_edges = np.stack([canny_edges, canny_edges, canny_edges], axis=2)
        canny_image = Image.fromarray(canny_edges)

        # --- 4. DYNAMIC PROMPT ---
        dynamic_prompt = (
            f"high quality detailed image, sports car, "
            f"painted in {palette_str}, "
            f"color scheme of {palette_names[0]} and {palette_names[1]}, "
            f"realistic, 8k, cinematic lighting"
        )
        print(f"📝 Prompt: {dynamic_prompt}")

        # --- 5. GENERATE ---
        output = pipe(
            dynamic_prompt,
            negative_prompt="low quality, bad quality, blurry, rust, dirty",
            image=canny_image,
            num_inference_steps=30,
            guidance_scale=8.0
        ).images[0]

        # --- 6. VISUALIZE ---
        fig, axs = plt.subplots(1, 4, figsize=(20, 5))

        # Reference
        axs[0].imshow(original_image)
        axs[0].set_title("Uploaded Reference")
        axs[0].axis("off")

        # Palette
        palette_img = np.zeros((100, 400, 3), dtype=np.uint8)
        step = 400 // 4
        for i, color in enumerate(palette_rgb):
            palette_img[:, i*step:(i+1)*step] = color
        axs[1].imshow(palette_img)
        axs[1].set_title(f"Palette: {palette_str}")
        axs[1].axis("off")

        # Edges
        axs[2].imshow(canny_image)
        axs[2].set_title("Structure (Edges)")
        axs[2].axis("off")

        # Result
        axs[3].imshow(output)
        axs[3].set_title("Generated Car")
        axs[3].axis("off")

        plt.show()

    except Exception as e:
        print(f"❌ Error processing image: {e}")
        print(f"Debug Info - Type of uploader.value: {type(uploader.value)}")
        print(f"Debug Info - Value: {uploader.value}")

In [ ]:
import io
import cv2
import numpy as np
import ipywidgets as widgets
from IPython.display import display, clear_output
from PIL import Image
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt

# --- CONFIGURATION FOR BETTER STRUCTURE ---
# Guess Mode forces the model to follow the edges strictly
GUESS_MODE = True
CONTROL_SCALE = 2.0  # Increased from 1.0 to 2.0 to lock geometry

# --- HELPER FUNCTIONS (Same as before) ---
def get_palette(image_rgb, k=4):
    img_small = cv2.resize(image_rgb, (64, 64))
    pixels = img_small.reshape(-1, 3)
    kmeans = KMeans(n_clusters=k, n_init=10).fit(pixels)
    # Sort by frequency
    unique, counts = np.unique(kmeans.labels_, return_counts=True)
    sorted_indices = np.argsort(counts)[::-1]
    sorted_centers = kmeans.cluster_centers_[sorted_indices].astype(int)
    
    def name_color(rgb):
        # Basic euclidean distance to find color name
        colors = {
            "black": (0,0,0), "white": (255,255,255), "red": (255,0,0), "green": (0,128,0),
            "blue": (0,0,255), "yellow": (255,255,0), "cyan": (0,255,255), "magenta": (255,0,255),
            "silver": (192,192,192), "gray": (128,128,128), "maroon": (128,0,0),
            "olive": (128,128,0), "purple": (128,0,128), "teal": (0,128,128), "navy": (0,0,128)
        }
        min_dist = float("inf"); closest = "gray"
        for name, val in colors.items():
            dist = np.sqrt(np.sum((np.array(rgb)-np.array(val))**2))
            if dist < min_dist: min_dist = dist; closest = name
        return closest
        
    return sorted_centers, [name_color(c) for c in sorted_centers]

def decode_image(uploader_value):
    if isinstance(uploader_value, tuple): content = uploader_value[0]['content']
    else: key = list(uploader_value.keys())[0]; content = uploader_value[key]['content']
    return np.array(Image.open(io.BytesIO(content)).convert("RGB"))

# --- UI & LOGIC ---
btn_struct = widgets.FileUpload(accept='.jpg,.png, .jpeg', description="1. Shape Ref")
btn_color = widgets.FileUpload(accept='.jpg,.png, .jpeg', description="2. Color Ref")
btn_run = widgets.Button(description="🛠️ Fix Structure & Respray", button_style='danger')
out = widgets.Output()

def run_fix(b):
    with out:
        clear_output()
        if not btn_struct.value or not btn_color.value: print("⚠️ Upload both images."); return
        
        try:
            print("Processing... (Enforcing Geometry Lock)")
            img_struct = decode_image(btn_struct.value)
            img_color = decode_image(btn_color.value)

            # 1. Stronger Edge Detection (Lower thresholds catch more details)
            edges = cv2.Canny(img_struct, 50, 150) # Lowered from 100, 200
            edges_rgb = np.stack([edges]*3, axis=2)
            canny_pil = Image.fromarray(edges_rgb)

            # 2. Extract Colors
            palette_rgb, palette_names = get_palette(img_color, k=3)
            main_c, trim_c = palette_names[0], palette_names[1]
            print(f"🎨 Palette: {main_c} body, {trim_c} trim")

            # 3. Prompt Engineering
            # Important: Removed "vehicle" to stop it from imagining a default car. 
            # We describe the MATERIALS, relying on ControlNet for the OBJECT.
            prompt = (
                f"a photo of an object painted in {main_c} with {trim_c} details, "
                f"automotive paint finish, 8k, highly detailed, realistic texture"
            )

            # 4. Generate with GUESS MODE
            result = pipe(
                prompt,
                negative_prompt="blurry, low quality, distorted, cartoon",
                image=canny_pil,
                num_inference_steps=50,
                guidance_scale=9.0,
                controlnet_conditioning_scale=CONTROL_SCALE,
                guess_mode=GUESS_MODE 
            ).images[0]

            # 5. Visualize
            fig, axs = plt.subplots(1, 3, figsize=(18, 6))
            axs[0].imshow(img_struct); axs[0].set_title("Input Shape"); axs[0].axis("off")
            axs[1].imshow(canny_pil); axs[1].set_title("Control Signal (Edges)"); axs[1].axis("off")
            axs[2].imshow(result); axs[2].set_title("Result (Structure Locked)"); axs[2].axis("off")
            plt.show()

        except Exception as e: print(f"Error: {e}")

btn_run.on_click(run_fix)
display(widgets.HBox([btn_struct, btn_color]), btn_run, out)

In [ ]:
import os
import cv2
import gc
import random
import numpy as np
import torch
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.cuda.amp import autocast, GradScaler # Native PyTorch Mixed Precision
from diffusers import ControlNetModel, DDPMScheduler, AutoencoderKL, UNet2DConditionModel
from transformers import AutoTokenizer, CLIPTextModel
from tqdm.auto import tqdm

# --- 1. HARD RESET & CONFIG ---
torch.cuda.empty_cache()
gc.collect()

# Config
MODEL_ID = "runwayml/stable-diffusion-v1-5"
OUTPUT_DIR = "/kaggle/working/controlnet_model_robust"
DATA_PATH = "/kaggle/input/processed-images/content/drive/MyDrive/ControlNet_Project/data/processed"
NUM_EPOCHS = 5
BATCH_SIZE = 2 # Reduced to 2 to be 100% safe on VRAM
LEARNING_RATE = 1e-5
DEVICE = "cuda"

if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR)

# --- 2. DATASET ---
class RobustDataset(Dataset):
    def __init__(self, data_root, tokenizer, size=512):
        self.data_root = data_root
        self.tokenizer = tokenizer
        self.size = size
        self.paths = []
        for root, _, files in os.walk(data_root):
            for f in files:
                if f.lower().endswith(('.jpg', '.jpeg', '.png')):
                    self.paths.append(os.path.join(root, f))
        print(f"📚 Loaded {len(self.paths)} images.")

    def __len__(self): return len(self.paths)

    def __getitem__(self, idx):
        path = self.paths[idx]
        img = cv2.imread(path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        
        # Random Edge Detection (Robustness)
        low = random.randint(50, 100)
        high = random.randint(150, 250)
        edges = cv2.Canny(img, low, high)
        edges = np.stack([edges]*3, axis=2)

        # Resize
        img = cv2.resize(img, (self.size, self.size))
        edges = cv2.resize(edges, (self.size, self.size))

        # Norm
        target = (img.astype(np.float32) / 127.5) - 1.0
        control = edges.astype(np.float32) / 255.0

        # Empty Prompt Training (50% chance)
        prompt = "" if random.random() < 0.5 else "high quality detailed car image"
        
        inputs = self.tokenizer(
            prompt, max_length=77, padding="max_length", truncation=True, return_tensors="pt"
        )

        return {
            "pixel_values": torch.from_numpy(target).permute(2,0,1),
            "control_values": torch.from_numpy(control).permute(2,0,1),
            "input_ids": inputs.input_ids[0]
        }

# --- 3. LOAD MODELS (MANUAL CASTING) ---
print("⏳ Loading models...")
tokenizer = AutoTokenizer.from_pretrained(MODEL_ID, subfolder="tokenizer")
noise_scheduler = DDPMScheduler.from_pretrained(MODEL_ID, subfolder="scheduler")
text_encoder = CLIPTextModel.from_pretrained(MODEL_ID, subfolder="text_encoder")
vae = AutoencoderKL.from_pretrained(MODEL_ID, subfolder="vae")
unet = UNet2DConditionModel.from_pretrained(MODEL_ID, subfolder="unet")

# Create ControlNet
controlnet = ControlNetModel.from_unet(unet)

# --- 4. DEVICE PLACEMENT (CRITICAL STEP) ---
# Freeze Base Models & Move to FP16 (Half Precision)
vae.requires_grad_(False)
unet.requires_grad_(False)
text_encoder.requires_grad_(False)

vae.to(DEVICE, dtype=torch.float16)
unet.to(DEVICE, dtype=torch.float16)
text_encoder.to(DEVICE, dtype=torch.float16)

# Keep ControlNet Trainable & in FP32 (Full Precision) for stability
controlnet.train()
controlnet.to(DEVICE, dtype=torch.float32) 
controlnet.enable_gradient_checkpointing()

# --- 5. OPTIMIZER & LOADER ---
optimizer = torch.optim.AdamW(controlnet.parameters(), lr=LEARNING_RATE)
dataset = RobustDataset(DATA_PATH, tokenizer)
dataloader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=2)
scaler = GradScaler() # Handles the mixed precision math

# --- 6. RAW TRAINING LOOP ---
print("🚀 Starting Training (Raw PyTorch Mode)...")

for epoch in range(NUM_EPOCHS):
    pbar = tqdm(dataloader, desc=f"Epoch {epoch+1}/{NUM_EPOCHS}")
    
    for batch in pbar:
        optimizer.zero_grad()
        
        # 1. Prepare Data
        # Base images -> FP16 (matches VAE)
        pixel_values = batch["pixel_values"].to(DEVICE, dtype=torch.float16)
        # Control images -> FP16 (matches ControlNet input expectation in autocast)
        control_values = batch["control_values"].to(DEVICE, dtype=torch.float16)
        input_ids = batch["input_ids"].to(DEVICE)

        # 2. Forward Pass (Mixed Precision Context)
        with autocast(dtype=torch.float16):
            # Encode Latents
            latents = vae.encode(pixel_values).latent_dist.sample()
            latents = latents * vae.config.scaling_factor

            # Encode Text
            encoder_hidden_states = text_encoder(input_ids)[0]

            # Noise
            noise = torch.randn_like(latents)
            timesteps = torch.randint(0, noise_scheduler.config.num_train_timesteps, (latents.shape[0],), device=DEVICE).long()
            noisy_latents = noise_scheduler.add_noise(latents, noise, timesteps)

            # ControlNet
            down_res, mid_res = controlnet(
                noisy_latents,
                timesteps,
                encoder_hidden_states=encoder_hidden_states,
                controlnet_cond=control_values,
                return_dict=False,
            )

            # UNet
            noise_pred = unet(
                noisy_latents,
                timesteps,
                encoder_hidden_states=encoder_hidden_states,
                down_block_additional_residuals=down_res,
                mid_block_additional_residual=mid_res,
            ).sample

            # Loss
            loss = F.mse_loss(noise_pred.float(), noise.float(), reduction="mean")

        # 3. Backward (Scaled)
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        
        pbar.set_postfix(loss=loss.item())

    # Save
    save_path = os.path.join(OUTPUT_DIR, f"checkpoint-epoch-{epoch+1}")
    controlnet.save_pretrained(save_path)
    print(f"💾 Saved: {save_path}")

print("✅ Training Complete.")

In [ ]:
import gc
import torch

# 1. Delete heavy training variables if they exist
# We use 'globals()' to find them safely without crashing if they don't exist
variables_to_delete = ['controlnet', 'unet', 'vae', 'text_encoder', 'optimizer', 'accelerator', 'pipeline', 'pipe']

for var in variables_to_delete:
    if var in globals():
        del globals()[var]

# 2. Force Garbage Collection
gc.collect()

# 3. Clear CUDA Cache
torch.cuda.empty_cache()

# 4. Verify Memory
if torch.cuda.is_available():
    free_mem = torch.cuda.mem_get_info()[0] / 1e9
    total_mem = torch.cuda.mem_get_info()[1] / 1e9
    print(f"✅ VRAM Cleared! Free Memory: {free_mem:.2f} GB / {total_mem:.2f} GB")
else:
    print("⚠️ No GPU detected.")

In [ ]:
import os
import cv2
import torch
import random
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
from diffusers import StableDiffusionControlNetPipeline, ControlNetModel, UniPCMultistepScheduler

# --- 1. CONFIGURATION ---
# Ensure this matches your saved path
CHECKPOINT_PATH = "/kaggle/working/controlnet_model_robust/checkpoint-epoch-5" 
BASE_MODEL = "runwayml/stable-diffusion-v1-5"
DATA_ROOT = "/kaggle/input/processed-images/content/drive/MyDrive/ControlNet_Project/data/processed"

# --- 2. MEMORY OPTIMIZED LOADING ---
print("⏳ Loading Pipeline in Low-VRAM Mode...")

# Load ControlNet in FP16
controlnet = ControlNetModel.from_pretrained(
    CHECKPOINT_PATH, 
    torch_dtype=torch.float16
)

# Load Pipeline in FP16
pipe = StableDiffusionControlNetPipeline.from_pretrained(
    BASE_MODEL,
    controlnet=controlnet,
    torch_dtype=torch.float16,
    safety_checker=None
)

# CRITICAL MEMORY SETTINGS
pipe.scheduler = UniPCMultistepScheduler.from_config(pipe.scheduler.config)
pipe.enable_model_cpu_offload() # Auto-moves parts to CPU when not used
pipe.enable_vae_slicing()       # Decodes image in chunks (Saves ~2GB VRAM)

print("✅ Model Loaded successfully!")

def create_overlay(generated_img, edge_map):
    # 1. Convert Generated Image to Grayscale & back to RGB
    gen_gray = cv2.cvtColor(np.array(generated_img), cv2.COLOR_RGB2GRAY)
    gen_gray = cv2.cvtColor(gen_gray, cv2.COLOR_GRAY2RGB)
    
    # 2. Prepare Edge Map
    edge_data = np.array(edge_map)
    if len(edge_data.shape) == 3:
        edge_data = edge_data[:,:,0] # Take one channel if it's 3-channel
    
    # --- FIX: RESIZE EDGES TO MATCH GENERATED IMAGE ---
    # This prevents the shape mismatch error (265 vs 264)
    if edge_data.shape[:2] != gen_gray.shape[:2]:
        edge_data = cv2.resize(edge_data, (gen_gray.shape[1], gen_gray.shape[0]), interpolation=cv2.INTER_NEAREST)

    # 3. Create Green Mask
    green_mask = np.zeros_like(gen_gray)
    green_mask[:, :, 1] = edge_data 
    
    # 4. Blend
    return cv2.addWeighted(gen_gray, 0.5, green_mask, 1.0, 0)

# --- 4. EXECUTION ---
# Pick random image
all_images = [os.path.join(r, f) for r, _, fs in os.walk(DATA_ROOT) for f in fs if f.lower().endswith(('.jpg', '.png'))]
if not all_images:
    raise FileNotFoundError("Could not find images. Check DATA_ROOT path.")

random_path = random.choice(all_images)
original_img = Image.open(random_path).convert("RGB")
original_cv2 = np.array(original_img)

# Extract Canny
edges = cv2.Canny(original_cv2, 100, 200)
edges_rgb = np.stack([edges]*3, axis=2)
canny_pil = Image.fromarray(edges_rgb)

print("🧪 Generating...")

# Test 1: Blind Structure Check (Empty Prompt)
# This proves if the model learned the SHAPE, not just the concept.
output_blind = pipe(
    prompt="", 
    negative_prompt="blurry, distorted, low quality, artifact",
    image=canny_pil,
    num_inference_steps=25, # Reduced steps for speed/safety
    guidance_scale=4.0,
).images[0]

# Test 2: Style Check
output_styled = pipe(
    prompt="futuristic sports car, neon lighting, cyberpunk city, 8k uhd, masterpiece", 
    negative_prompt="blurry, distorted, low quality, rust",
    image=canny_pil,
    num_inference_steps=25,
    guidance_scale=7.5,
).images[0]

# --- 5. VISUALIZE ---
fig, axs = plt.subplots(1, 4, figsize=(20, 5))

axs[0].imshow(canny_pil)
axs[0].set_title("Input Edges")
axs[0].axis("off")

axs[1].imshow(output_blind)
axs[1].set_title("Structure Check (No Prompt)")
axs[1].axis("off")

axs[2].imshow(create_overlay(output_blind, canny_pil))
axs[2].set_title("Alignment (Green = Input)")
axs[2].axis("off")

axs[3].imshow(output_styled)
axs[3].set_title("Style Test (Cyberpunk)")
axs[3].axis("off")

plt.show()

In [ ]:
import os
import cv2
import torch
import random
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
from diffusers import StableDiffusionControlNetPipeline, ControlNetModel, UniPCMultistepScheduler

# --- 1. CONFIGURATION ---
CHECKPOINT_PATH = "/kaggle/working/controlnet_model_robust/checkpoint-epoch-5" 
BASE_MODEL = "runwayml/stable-diffusion-v1-5"
DATA_ROOT = "/kaggle/input/processed-images/content/drive/MyDrive/ControlNet_Project/data/processed"

# --- 2. LOAD PIPELINE (Low VRAM Mode) ---
print("⏳ Loading Pipeline...")
controlnet = ControlNetModel.from_pretrained(CHECKPOINT_PATH, torch_dtype=torch.float16)
pipe = StableDiffusionControlNetPipeline.from_pretrained(
    BASE_MODEL, controlnet=controlnet, torch_dtype=torch.float16, safety_checker=None
)
pipe.scheduler = UniPCMultistepScheduler.from_config(pipe.scheduler.config)
pipe.enable_model_cpu_offload()

# --- 3. ROBUST OVERLAY FUNCTION (Crash Fix) ---
def create_safe_overlay(generated_img, edge_map):
    # Convert Gen to Grayscale -> RGB
    gen_gray = cv2.cvtColor(np.array(generated_img), cv2.COLOR_RGB2GRAY)
    gen_gray = cv2.cvtColor(gen_gray, cv2.COLOR_GRAY2RGB)
    
    # Prepare Edges
    edge_data = np.array(edge_map)
    if len(edge_data.shape) == 3: edge_data = edge_data[:,:,0]

    # FIX: Force resize edges to match generated image EXACTLY
    # This solves the 265 vs 264 pixel crash
    edge_resized = cv2.resize(edge_data, (gen_gray.shape[1], gen_gray.shape[0]), interpolation=cv2.INTER_NEAREST)
    
    green_mask = np.zeros_like(gen_gray)
    green_mask[:, :, 1] = edge_resized
    return cv2.addWeighted(gen_gray, 0.5, green_mask, 1.0, 0)

# --- 4. PICK RANDOM IMAGE ---
all_images = [os.path.join(r, f) for r, _, fs in os.walk(DATA_ROOT) for f in fs if f.lower().endswith(('.jpg', '.png'))]
if not all_images:
    raise FileNotFoundError("No images found! Check your DATA_ROOT path.")

random_path = random.choice(all_images)
print(f"🎲 Selected Image: {os.path.basename(random_path)}")

# Load & Resize (Standardizing to 512x512 helps stability)
original_img = Image.open(random_path).convert("RGB").resize((512, 512))
original_cv2 = np.array(original_img)

# Extract Canny
edges = cv2.Canny(original_cv2, 100, 200)
edges_rgb = np.stack([edges]*3, axis=2)
canny_pil = Image.fromarray(edges_rgb)

# --- 5. RUN STRICT INFERENCE ---
print("🧪 Running Strict Mode Inference...")

# We lower guidance (text strength) and raise controlnet strength
output = pipe(
    prompt="high quality masterpiece, a vehicle, cinematic lighting, 8k", 
    negative_prompt="blurry, distorted, low quality, illustration, cartoon",
    image=canny_pil,
    num_inference_steps=30,
    guidance_scale=4.0,              # Lowered to reduce text hallucinations
    controlnet_conditioning_scale=1.5 # Raised to force structure
).images[0]

# --- 6. VISUALIZE ---
fig, axs = plt.subplots(1, 3, figsize=(18, 6))

axs[0].imshow(canny_pil)
axs[0].set_title("Input Edges")
axs[0].axis("off")

axs[1].imshow(output)
axs[1].set_title("Result (Strict Mode)")
axs[1].axis("off")

# Apply the safe overlay
overlay = create_safe_overlay(output, canny_pil)
axs[2].imshow(overlay)
axs[2].set_title("Alignment Check\n(Green lines should match car)")
axs[2].axis("off")

plt.show()

In [ ]:
import io
import cv2
import numpy as np
import ipywidgets as widgets
from IPython.display import display, clear_output
from PIL import Image
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt

# --- 1. CONFIGURATION ---
# We keep the "Strict Mode" settings that worked
GUIDANCE_SCALE = 5.0       # Slightly higher to push color, but low enough to keep shape
CONTROL_SCALE = 1.6        # High strength to lock geometry
INFERENCE_STEPS = 40

# --- 2. COLOR EXTRACTOR ---
def get_palette(image_rgb, k=4):
    img_small = cv2.resize(image_rgb, (64, 64))
    pixels = img_small.reshape(-1, 3)
    kmeans = KMeans(n_clusters=k, n_init=10).fit(pixels)
    
    unique, counts = np.unique(kmeans.labels_, return_counts=True)
    sorted_indices = np.argsort(counts)[::-1]
    sorted_centers = kmeans.cluster_centers_[sorted_indices].astype(int)
    
    # Internal simple color namer
    colors = {
        "black": (0,0,0), "white": (255,255,255), "red": (255,0,0), "green": (0,128,0),
        "blue": (0,0,255), "yellow": (255,255,0), "silver": (192,192,192), 
        "gray": (128,128,128), "orange": (255,165,0), "gold": (255,215,0), 
        "purple": (128,0,128), "navy": (0,0,128), "teal": (0,128,128)
    }
    def name_c(rgb):
        return min(colors, key=lambda k: np.linalg.norm(np.array(rgb)-np.array(colors[k])))
    
    return sorted_centers, [name_c(c) for c in sorted_centers]

def decode_image(uploader_value):
    if isinstance(uploader_value, tuple): content = uploader_value[0]['content']
    else: key = list(uploader_value.keys())[0]; content = uploader_value[key]['content']
    return np.array(Image.open(io.BytesIO(content)).convert("RGB"))

# --- 3. THE APP ---
btn_struct = widgets.FileUpload(accept='.jpg,.png, .jpeg', description="1. Shape Ref")
btn_color = widgets.FileUpload(accept='.jpg,.png, .jpeg', description="2. Color Ref")
btn_run = widgets.Button(description="🎨 Respray Car", button_style='success', layout=widgets.Layout(width='300px'))
out = widgets.Output()

def run_respray(b):
    with out:
        clear_output()
        if not btn_struct.value or not btn_color.value: print("⚠️ Upload both images."); return
        
        try:
            print("⏳ Processing...")
            img_struct = decode_image(btn_struct.value)
            img_color = decode_image(btn_color.value)

            # 1. Edges (Structure)
            # Resize strict to 512 for stability
            img_struct_512 = cv2.resize(img_struct, (512, 512))
            edges = cv2.Canny(img_struct_512, 100, 200)
            edges_rgb = np.stack([edges]*3, axis=2)
            canny_pil = Image.fromarray(edges_rgb)

            # 2. Colors (Palette)
            palette_rgb, palette_names = get_palette(img_color, k=3)
            main_c, trim_c = palette_names[0], palette_names[1]
            print(f"🎨 Applying Paint: {main_c.upper()} Body with {trim_c.upper()} Trim")

            # 3. Dynamic Prompt
            # We use "vehicle" instead of "car" to avoid bias, but add "automotive paint" for texture
            prompt = (
                f"high quality photo of a vehicle painted in {main_c} with {trim_c} details, "
                f"automotive paint finish, highly polished, studio lighting, 8k, realistic"
            )

            # 4. Generate (Strict Mode)
            result = pipe(
                prompt,
                negative_prompt="cartoon, drawing, sketch, flat color, blurry, low quality",
                image=canny_pil,
                num_inference_steps=INFERENCE_STEPS,
                guidance_scale=GUIDANCE_SCALE,        # 5.0 (Balanced)
                controlnet_conditioning_scale=CONTROL_SCALE # 1.6 (Strict)
            ).images[0]

            # 5. Show Result
            fig, axs = plt.subplots(1, 3, figsize=(18, 6))
            axs[0].imshow(img_struct_512); axs[0].set_title("Target Shape"); axs[0].axis("off")
            
            # Show color source with palette bar
            p_bar = np.zeros((50, 300, 3), dtype=np.uint8)
            step = 300//3
            for i in range(3): p_bar[:, i*step:(i+1)*step] = palette_rgb[i]
            axs[1].imshow(img_color); axs[1].set_title(f"Target Color ({main_c})"); axs[1].axis("off")
            
            axs[2].imshow(result); axs[2].set_title("Final Respray"); axs[2].axis("off")
            plt.show()

        except Exception as e: print(f"Error: {e}")

btn_run.on_click(run_respray)
display(widgets.HBox([btn_struct, btn_color]))
display(btn_run)
display(out)

In [ ]:
import io
import cv2
import numpy as np
import ipywidgets as widgets
from IPython.display import display, clear_output
from PIL import Image
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import torch

# --- 1. CONFIGURATION (TUNED) ---
# We increased Guidance to force color, and lowered Control slightly to stop the "Sketch" look
GUIDANCE_SCALE = 8.0       # Was 5.0 -> Boosted to force the paint color
CONTROL_SCALE = 1.1        # Was 1.6 -> Lowered to stop it looking like a wireframe
INFERENCE_STEPS = 40

# Paths
CHECKPOINT_PATH = "/kaggle/working/controlnet_model_robust/checkpoint-epoch-5" 
BASE_MODEL = "runwayml/stable-diffusion-v1-5"

# --- 2. LOAD PIPELINE ---
controlnet = ControlNetModel.from_pretrained(CHECKPOINT_PATH, torch_dtype=torch.float16)
pipe = StableDiffusionControlNetPipeline.from_pretrained(
    BASE_MODEL, controlnet=controlnet, torch_dtype=torch.float16, safety_checker=None
)
pipe.scheduler = UniPCMultistepScheduler.from_config(pipe.scheduler.config)
pipe.enable_model_cpu_offload()

# --- 3. HELPER FUNCTIONS ---
def get_palette(image_rgb, k=4):
    img_small = cv2.resize(image_rgb, (64, 64))
    pixels = img_small.reshape(-1, 3)
    kmeans = KMeans(n_clusters=k, n_init=10).fit(pixels)
    
    unique, counts = np.unique(kmeans.labels_, return_counts=True)
    sorted_indices = np.argsort(counts)[::-1]
    sorted_centers = kmeans.cluster_centers_[sorted_indices].astype(int)
    
    colors = {
        "black": (0,0,0), "white": (255,255,255), "red": (255,0,0), "green": (0,128,0),
        "blue": (0,0,255), "yellow": (255,255,0), "silver": (192,192,192), 
        "gray": (128,128,128), "orange": (255,165,0), "gold": (255,215,0), 
        "purple": (128,0,128), "navy": (0,0,128), "teal": (0,128,128)
    }
    def name_c(rgb):
        return min(colors, key=lambda k: np.linalg.norm(np.array(rgb)-np.array(colors[k])))
    
    return sorted_centers, [name_c(c) for c in sorted_centers]

def decode_image(uploader_value):
    if isinstance(uploader_value, tuple): content = uploader_value[0]['content']
    else: key = list(uploader_value.keys())[0]; content = uploader_value[key]['content']
    return np.array(Image.open(io.BytesIO(content)).convert("RGB"))

# --- 4. THE APP UI ---
btn_struct = widgets.FileUpload(accept='.jpg,.png, .jpeg', description="1. Shape Ref")
btn_color = widgets.FileUpload(accept='.jpg,.png, .jpeg', description="2. Color Ref")
btn_run = widgets.Button(description="🎨 Respray Car", button_style='primary', layout=widgets.Layout(width='300px'))
out = widgets.Output()

def run_respray(b):
    with out:
        clear_output()
        if not btn_struct.value or not btn_color.value: print("⚠️ Upload both images."); return
        
        try:
            print("⏳ Processing...")
            img_struct = decode_image(btn_struct.value)
            img_color = decode_image(btn_color.value)

            # 1. Structure
            img_struct_512 = cv2.resize(img_struct, (512, 512))
            edges = cv2.Canny(img_struct_512, 100, 200)
            edges_rgb = np.stack([edges]*3, axis=2)
            canny_pil = Image.fromarray(edges_rgb)

            # 2. Colors
            palette_rgb, palette_names = get_palette(img_color, k=3)
            main_c, trim_c = palette_names[0], palette_names[1]
            print(f"🎨 Palette: {main_c.upper()} / {trim_c.upper()}")

            # 3. Dynamic Prompt (ENFORCED REALISM)
            # We add "photograph" and "matte paint" to stop the wireframe look
            prompt = (
                f"raw photograph of a car painted in {main_c}, with {trim_c} details, "
                f"automotive paint, realistic texture, 8k uhd, highly detailed, "
                f"matching color palette: {main_c} and {trim_c}"
            )

            # 4. Generate
            result = pipe(
                prompt,
                negative_prompt="sketch, drawing, wireframe, greyscale, cartoon, illustration, low quality",
                image=canny_pil,
                num_inference_steps=INFERENCE_STEPS,
                guidance_scale=GUIDANCE_SCALE,        # 8.0 (Forces Color)
                controlnet_conditioning_scale=CONTROL_SCALE # 1.1 (Allows Paint)
            ).images[0]

            # 5. Show
            fig, axs = plt.subplots(1, 3, figsize=(18, 6))
            axs[0].imshow(img_struct_512); axs[0].set_title("Shape"); axs[0].axis("off")
            axs[1].imshow(img_color); axs[1].set_title(f"Colors ({main_c}/{trim_c})"); axs[1].axis("off")
            axs[2].imshow(result); axs[2].set_title("Final Respray"); axs[2].axis("off")
            plt.show()

        except Exception as e: print(f"Error: {e}")

btn_run.on_click(run_respray)
display(widgets.HBox([btn_struct, btn_color]))
display(btn_run)
display(out)

In [ ]:
import io
import cv2
import numpy as np
import ipywidgets as widgets
from IPython.display import display, clear_output
from PIL import Image
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import torch

# --- 1. CONFIGURATION (TUNED) ---
# We increased Guidance to force color, and lowered Control slightly to stop the "Sketch" look
GUIDANCE_SCALE = 8.0       # Was 5.0 -> Boosted to force the paint color
CONTROL_SCALE = 1.1        # Was 1.6 -> Lowered to stop it looking like a wireframe
INFERENCE_STEPS = 40

# Paths
CHECKPOINT_PATH = "/kaggle/working/controlnet_model_robust/checkpoint-epoch-5" 
BASE_MODEL = "runwayml/stable-diffusion-v1-5"

# --- 2. LOAD PIPELINE ---
# Load optimized pipeline
controlnet = ControlNetModel.from_pretrained(CHECKPOINT_PATH, torch_dtype=torch.float16)
pipe = StableDiffusionControlNetPipeline.from_pretrained(
    BASE_MODEL, controlnet=controlnet, torch_dtype=torch.float16, safety_checker=None
)
pipe.scheduler = UniPCMultistepScheduler.from_config(pipe.scheduler.config)
pipe.enable_model_cpu_offload()

# --- 3. HELPER FUNCTIONS ---
def get_palette(image_rgb, k=4):
    img_small = cv2.resize(image_rgb, (64, 64))
    pixels = img_small.reshape(-1, 3)
    kmeans = KMeans(n_clusters=k, n_init=10).fit(pixels)
    
    unique, counts = np.unique(kmeans.labels_, return_counts=True)
    sorted_indices = np.argsort(counts)[::-1]
    sorted_centers = kmeans.cluster_centers_[sorted_indices].astype(int)
    
    colors = {
        "black": (0,0,0), "white": (255,255,255), "red": (255,0,0), "green": (0,128,0),
        "blue": (0,0,255), "yellow": (255,255,0), "silver": (192,192,192), 
        "gray": (128,128,128), "orange": (255,165,0), "gold": (255,215,0), 
        "purple": (128,0,128), "navy": (0,0,128), "teal": (0,128,128)
    }
    def name_c(rgb):
        return min(colors, key=lambda k: np.linalg.norm(np.array(rgb)-np.array(colors[k])))
    
    return sorted_centers, [name_c(c) for c in sorted_centers]

def decode_image(uploader_value):
    if isinstance(uploader_value, tuple): content = uploader_value[0]['content']
    else: key = list(uploader_value.keys())[0]; content = uploader_value[key]['content']
    return np.array(Image.open(io.BytesIO(content)).convert("RGB"))

# --- 4. THE APP UI ---
btn_struct = widgets.FileUpload(accept='.jpg,.png, .jpeg', description="1. Shape Ref")
btn_color = widgets.FileUpload(accept='.jpg,.png, .jpeg', description="2. Color Ref")
btn_run = widgets.Button(description="🎨 Respray Car", button_style='primary', layout=widgets.Layout(width='300px'))
out = widgets.Output()

def run_respray(b):
    with out:
        clear_output()
        if not btn_struct.value or not btn_color.value: print("⚠️ Upload both images."); return
        
        try:
            print("⏳ Processing...")
            img_struct = decode_image(btn_struct.value)
            img_color = decode_image(btn_color.value)

            # 1. Structure
            img_struct_512 = cv2.resize(img_struct, (512, 512))
            edges = cv2.Canny(img_struct_512, 100, 200)
            edges_rgb = np.stack([edges]*3, axis=2)
            canny_pil = Image.fromarray(edges_rgb)

            # 2. Colors
            palette_rgb, palette_names = get_palette(img_color, k=3)
            main_c, trim_c = palette_names[0], palette_names[1]
            print(f"🎨 Palette: {main_c.upper()} / {trim_c.upper()}")

            # 3. Dynamic Prompt (ENFORCED REALISM)
            # We add "photograph" and "matte paint" to stop the wireframe look
            prompt = (
                f"raw photograph of a car painted in {main_c}, with {trim_c} details, "
                f"automotive paint, realistic texture, 8k uhd, highly detailed, "
                f"matching color palette: {main_c} and {trim_c}"
            )

            # 4. Generate
            result = pipe(
                prompt,
                negative_prompt="sketch, drawing, wireframe, greyscale, cartoon, illustration, low quality",
                image=canny_pil,
                num_inference_steps=INFERENCE_STEPS,
                guidance_scale=GUIDANCE_SCALE,        # 8.0 (Forces Color)
                controlnet_conditioning_scale=CONTROL_SCALE # 1.1 (Allows Paint)
            ).images[0]

            # 5. Show
            fig, axs = plt.subplots(1, 3, figsize=(18, 6))
            axs[0].imshow(img_struct_512); axs[0].set_title("Shape"); axs[0].axis("off")
            axs[1].imshow(img_color); axs[1].set_title(f"Colors ({main_c}/{trim_c})"); axs[1].axis("off")
            axs[2].imshow(result); axs[2].set_title("Final Respray"); axs[2].axis("off")
            plt.show()

        except Exception as e: print(f"Error: {e}")

btn_run.on_click(run_respray)
display(widgets.HBox([btn_struct, btn_color]))
display(btn_run)
display(out)

In [ ]:
import os
import cv2
import gc
import random
import numpy as np
import torch
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.cuda.amp import autocast, GradScaler
from diffusers import ControlNetModel, DDPMScheduler, AutoencoderKL, UNet2DConditionModel
from transformers import AutoTokenizer, CLIPTextModel
from tqdm.auto import tqdm
from scipy.spatial import KDTree

# --- 1. MEMORY RESET ---
torch.cuda.empty_cache()
gc.collect()

# --- 2. CONFIGURATION ---
MODEL_ID = "runwayml/stable-diffusion-v1-5"
OUTPUT_DIR = "/kaggle/working/controlnet_model_final_color"

# PATHS
# 1. The "Index" (Defines WHICH images to use)
PROCESSED_PATH = "/kaggle/input/processed-images/content/drive/MyDrive/ControlNet_Project/data/processed"
# 2. The "Source" (Where the COLOR comes from)
RAW_PATH = "/kaggle/input/processed-images/content/drive/MyDrive/ControlNet_Project/data/raw"

NUM_EPOCHS = 5
BATCH_SIZE = 4
LEARNING_RATE = 1e-5
DEVICE = "cuda"

if not os.path.exists(OUTPUT_DIR): os.makedirs(OUTPUT_DIR)

# --- 3. DUAL-PATH DATASET ---
class DualPathDataset(Dataset):
    def __init__(self, processed_root, raw_root, tokenizer, size=512):
        self.processed_root = processed_root
        self.raw_root = raw_root
        self.tokenizer = tokenizer
        self.size = size
        self.valid_files = []
        
        print(f"🔍 Scanning Index: {processed_root}")
        
        # 1. Build the list based on PROCESSED folder (The Subset)
        for root, _, files in os.walk(processed_root):
            for f in files:
                if f.lower().endswith(('.jpg', '.png', '.jpeg')):
                    # Check if this file exists in RAW before adding
                    raw_p = os.path.join(raw_root, f)
                    if os.path.exists(raw_p):
                        self.valid_files.append(f) # Store filename only
                    
        print(f"📚 Dataset Ready: {len(self.valid_files)} matched images.")
        if len(self.valid_files) > 0:
            print(f"✅ Example match: {self.valid_files[0]} exists in both folders.")

        # Color Lookup
        self.color_map = {
            "black": (0,0,0), "white": (255,255,255), "red": (255,0,0),
            "green": (0,128,0), "blue": (0,0,255), "yellow": (255,255,0),
            "silver": (192,192,192), "gray": (128,128,128), "orange": (255,165,0),
            "purple": (128,0,128), "brown": (165,42,42), "beige": (245,245,220),
            "gold": (255,215,0), "teal": (0,128,128), "navy": (0,0,128)
        }
        self.color_names = list(self.color_map.keys())
        self.color_values = list(self.color_map.values())
        self.kdtree = KDTree(self.color_values)

    def get_dominant_color(self, image_bgr):
        small = cv2.resize(image_bgr, (64, 64))
        small_rgb = cv2.cvtColor(small, cv2.COLOR_BGR2RGB)
        h, w, _ = small_rgb.shape
        center = small_rgb[h//4:h*3//4, w//4:w*3//4]
        mean_color = center.mean(axis=(0,1))
        dist, idx = self.kdtree.query(mean_color)
        return self.color_names[idx]

    def __len__(self): return len(self.valid_files)

    def __getitem__(self, idx):
        filename = self.valid_files[idx]
        
        # CRITICAL: Load from RAW folder to get the Color
        raw_path = os.path.join(self.raw_root, filename)
        img_bgr = cv2.imread(raw_path)
        
        # Safety check for broken images
        if img_bgr is None:
            # Fallback to random black image to prevent crash
            img_bgr = np.zeros((512,512,3), dtype=np.uint8)
        
        img_rgb = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2RGB)
        
        # 1. Extract Color
        color_name = self.get_dominant_color(img_bgr)
        
        # 2. Generate Edges (On the fly from Raw)
        low = random.randint(50, 100)
        high = random.randint(150, 250)
        edges = cv2.Canny(img_bgr, low, high)
        edges_rgb = np.stack([edges]*3, axis=2)

        # 3. Resize & Norm
        img_rgb = cv2.resize(img_rgb, (self.size, self.size))
        edges_rgb = cv2.resize(edges_rgb, (self.size, self.size))

        target = (img_rgb.astype(np.float32) / 127.5) - 1.0
        control = edges_rgb.astype(np.float32) / 255.0

        # 4. Prompt Logic
        if random.random() < 0.5:
            prompt = "" 
        else:
            prompt = f"a {color_name} car, automotive paint"

        inputs = self.tokenizer(
            prompt, max_length=77, padding="max_length", truncation=True, return_tensors="pt"
        )

        return {
            "pixel_values": torch.from_numpy(target).permute(2,0,1),
            "control_values": torch.from_numpy(control).permute(2,0,1),
            "input_ids": inputs.input_ids[0]
        }

# --- 4. MODEL LOADING ---
print("⏳ Loading Models...")
tokenizer = AutoTokenizer.from_pretrained(MODEL_ID, subfolder="tokenizer")
noise_scheduler = DDPMScheduler.from_pretrained(MODEL_ID, subfolder="scheduler")
text_encoder = CLIPTextModel.from_pretrained(MODEL_ID, subfolder="text_encoder")
vae = AutoencoderKL.from_pretrained(MODEL_ID, subfolder="vae")
unet = UNet2DConditionModel.from_pretrained(MODEL_ID, subfolder="unet")

controlnet = ControlNetModel.from_unet(unet)

vae.requires_grad_(False)
unet.requires_grad_(False)
text_encoder.requires_grad_(False)

vae.to(DEVICE, dtype=torch.float16)
unet.to(DEVICE, dtype=torch.float16)
text_encoder.to(DEVICE, dtype=torch.float16)

controlnet.to(DEVICE, dtype=torch.float32)
controlnet.train()
controlnet.enable_gradient_checkpointing()

# --- 5. TRAINING LOOP ---
optimizer = torch.optim.AdamW(controlnet.parameters(), lr=LEARNING_RATE)
dataset = DualPathDataset(PROCESSED_PATH, RAW_PATH, tokenizer)

# Check if dataset is empty before starting
if len(dataset) == 0:
    print("❌ ERROR: No matching files found! Check your RAW_PATH and PROCESSED_PATH.")
else:
    dataloader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=2)
    scaler = GradScaler()

    print(f"🚀 Starting Dual-Path Training for {NUM_EPOCHS} Epochs...")

    for epoch in range(NUM_EPOCHS):
        pbar = tqdm(dataloader, desc=f"Epoch {epoch+1}/{NUM_EPOCHS}")
        
        for batch in pbar:
            optimizer.zero_grad()
            
            pixel_values = batch["pixel_values"].to(DEVICE, dtype=torch.float16)
            control_values = batch["control_values"].to(DEVICE, dtype=torch.float16)
            input_ids = batch["input_ids"].to(DEVICE)

            with autocast(dtype=torch.float16):
                latents = vae.encode(pixel_values).latent_dist.sample() * vae.config.scaling_factor
                encoder_hidden_states = text_encoder(input_ids)[0]
                
                noise = torch.randn_like(latents)
                timesteps = torch.randint(0, noise_scheduler.config.num_train_timesteps, (latents.shape[0],), device=DEVICE).long()
                noisy_latents = noise_scheduler.add_noise(latents, noise, timesteps)

                down_res, mid_res = controlnet(
                    noisy_latents,
                    timesteps,
                    encoder_hidden_states=encoder_hidden_states,
                    controlnet_cond=control_values,
                    return_dict=False,
                )

                noise_pred = unet(
                    noisy_latents,
                    timesteps,
                    encoder_hidden_states=encoder_hidden_states,
                    down_block_additional_residuals=down_res,
                    mid_block_additional_residual=mid_res,
                ).sample

                loss = F.mse_loss(noise_pred.float(), noise.float(), reduction="mean")

            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()
            pbar.set_postfix(loss=loss.item())

        save_path = os.path.join(OUTPUT_DIR, f"checkpoint-epoch-{epoch+1}")
        controlnet.save_pretrained(save_path)
        print(f"💾 Saved: {save_path}")

    print("✅ Training Complete.")

In [ ]:
import os
import cv2
import gc
import random
import numpy as np
import torch
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.cuda.amp import autocast, GradScaler
from diffusers import ControlNetModel, DDPMScheduler, AutoencoderKL, UNet2DConditionModel
from transformers import AutoTokenizer, CLIPTextModel
from tqdm.auto import tqdm
from scipy.spatial import KDTree

# --- 1. MEMORY RESET ---
torch.cuda.empty_cache()
gc.collect()

# --- 2. CONFIGURATION ---
MODEL_ID = "runwayml/stable-diffusion-v1-5"
OUTPUT_DIR = "/kaggle/working/controlnet_model_final_1k"

# 🎯 NEW PATH (Raw Color Images)
DATA_PATH = "/kaggle/input/cars-colored/cars" 

NUM_EPOCHS = 5
BATCH_SIZE = 4
LEARNING_RATE = 1e-5
DEVICE = "cuda"

if not os.path.exists(OUTPUT_DIR): os.makedirs(OUTPUT_DIR)

# --- 3. DATASET CLASS (With 1000 Image Limit) ---
class LimitedColorDataset(Dataset):
    def __init__(self, data_root, tokenizer, size=512, limit=1000):
        self.tokenizer = tokenizer
        self.size = size
        self.paths = []
        
        print(f"🔍 Scanning: {data_root}")
        
        # 1. Collect all valid images
        all_files = []
        for root, _, files in os.walk(data_root):
            for f in files:
                if f.lower().endswith(('.jpg', '.png', '.jpeg')):
                    all_files.append(os.path.join(root, f))
        
        # 2. Sort to ensure consistent selection
        all_files.sort()
        
        # 3. Apply the Limit (First 1000)
        self.paths = all_files[:limit]
        
        print(f"📚 Total found: {len(all_files)}")
        print(f"✂️ Training on subset: {len(self.paths)} images")

        # Color Lookup Table
        self.color_map = {
            "black": (0,0,0), "white": (255,255,255), "red": (255,0,0),
            "green": (0,128,0), "blue": (0,0,255), "yellow": (255,255,0),
            "silver": (192,192,192), "gray": (128,128,128), "orange": (255,165,0),
            "purple": (128,0,128), "navy": (0,0,128), "gold": (255,215,0), "teal": (0,128,128)
        }
        self.color_names = list(self.color_map.keys())
        self.color_values = list(self.color_map.values())
        self.kdtree = KDTree(self.color_values)

    def get_color(self, img_bgr):
        small = cv2.resize(img_bgr, (64, 64))
        rgb = cv2.cvtColor(small, cv2.COLOR_BGR2RGB)
        h, w, _ = rgb.shape
        center = rgb[h//4:h*3//4, w//4:w*3//4]
        mean = center.mean(axis=(0,1))
        _, idx = self.kdtree.query(mean)
        return self.color_names[idx]

    def __len__(self): return len(self.paths)

    def __getitem__(self, idx):
        path = self.paths[idx]
        img_bgr = cv2.imread(path)
        
        # Safety check for corrupt images
        if img_bgr is None:
            img_bgr = np.zeros((512, 512, 3), dtype=np.uint8)

        # 1. Extract Color Label
        color_name = self.get_color(img_bgr)
        
        # 2. Generate Control Signal (Edges)
        low = random.randint(50, 100)
        high = random.randint(150, 250)
        edges = cv2.Canny(img_bgr, low, high)
        edges_rgb = np.stack([edges]*3, axis=2)
        
        # 3. Resize & Norm
        img_rgb = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2RGB)
        img_rgb = cv2.resize(img_rgb, (self.size, self.size))
        edges_rgb = cv2.resize(edges_rgb, (self.size, self.size))

        target = (img_rgb.astype(np.float32) / 127.5) - 1.0
        control = edges_rgb.astype(np.float32) / 255.0

        # 4. Prompt Logic
        if random.random() < 0.5:
            prompt = "" 
        else:
            prompt = f"a {color_name} car, automotive paint"

        inputs = self.tokenizer(prompt, max_length=77, padding="max_length", truncation=True, return_tensors="pt")

        return {
            "pixel_values": torch.from_numpy(target).permute(2,0,1),
            "control_values": torch.from_numpy(control).permute(2,0,1),
            "input_ids": inputs.input_ids[0]
        }

# --- 4. MODEL LOADING ---
print("⏳ Loading Pipeline...")
tokenizer = AutoTokenizer.from_pretrained(MODEL_ID, subfolder="tokenizer")
noise_scheduler = DDPMScheduler.from_pretrained(MODEL_ID, subfolder="scheduler")
text_encoder = CLIPTextModel.from_pretrained(MODEL_ID, subfolder="text_encoder")
vae = AutoencoderKL.from_pretrained(MODEL_ID, subfolder="vae")
unet = UNet2DConditionModel.from_pretrained(MODEL_ID, subfolder="unet")

controlnet = ControlNetModel.from_unet(unet)

# Freeze & Optimize
vae.requires_grad_(False); unet.requires_grad_(False); text_encoder.requires_grad_(False)
vae.to(DEVICE, dtype=torch.float16)
unet.to(DEVICE, dtype=torch.float16)
text_encoder.to(DEVICE, dtype=torch.float16)
controlnet.to(DEVICE, dtype=torch.float32) # FP32 for stability
controlnet.train()
controlnet.enable_gradient_checkpointing()

# --- 5. EXECUTE ---
optimizer = torch.optim.AdamW(controlnet.parameters(), lr=LEARNING_RATE)
dataset = LimitedColorDataset(DATA_PATH, tokenizer, limit=1000)

if len(dataset) == 0:
    print("❌ Error: No images found. Check path.")
else:
    dataloader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=2)
    scaler = GradScaler()
    print(f"🚀 Training on {len(dataset)} images...")

    for epoch in range(NUM_EPOCHS):
        pbar = tqdm(dataloader, desc=f"Epoch {epoch+1}/{NUM_EPOCHS}")
        for batch in pbar:
            optimizer.zero_grad()
            
            pixel_values = batch["pixel_values"].to(DEVICE, dtype=torch.float16)
            control_values = batch["control_values"].to(DEVICE, dtype=torch.float16)
            input_ids = batch["input_ids"].to(DEVICE)

            with autocast(dtype=torch.float16):
                latents = vae.encode(pixel_values).latent_dist.sample() * vae.config.scaling_factor
                encoder_hidden_states = text_encoder(input_ids)[0]
                noise = torch.randn_like(latents)
                timesteps = torch.randint(0, noise_scheduler.config.num_train_timesteps, (latents.shape[0],), device=DEVICE).long()
                noisy_latents = noise_scheduler.add_noise(latents, noise, timesteps)

                down_res, mid_res = controlnet(
                    noisy_latents,
                    timesteps,
                    encoder_hidden_states=encoder_hidden_states,
                    controlnet_cond=control_values,
                    return_dict=False
                )

                noise_pred = unet(
                    noisy_latents,
                    timesteps,
                    encoder_hidden_states=encoder_hidden_states,
                    down_block_additional_residuals=down_res,
                    mid_block_additional_residual=mid_res
                ).sample

                loss = F.mse_loss(noise_pred.float(), noise.float(), reduction="mean")

            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()
            pbar.set_postfix(loss=loss.item())

        save_path = os.path.join(OUTPUT_DIR, f"checkpoint-epoch-{epoch+1}")
        controlnet.save_pretrained(save_path)
        print(f"💾 Saved: {save_path}")
    print("✅ Training Complete.")

In [ ]:
import os
import cv2
import torch
import random
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
from diffusers import StableDiffusionControlNetPipeline, ControlNetModel, UniPCMultistepScheduler

# --- 1. CONFIGURATION ---
# Point to Epoch 3 as requested
CHECKPOINT_PATH = "/kaggle/working/controlnet_model_final_1k/checkpoint-epoch-3"
BASE_MODEL = "runwayml/stable-diffusion-v1-5"
DATA_ROOT = "/kaggle/input/cars-colored/cars"

# --- 2. LOAD PIPELINE ---
print(f"⏳ Loading Epoch 3 from: {CHECKPOINT_PATH}")
controlnet = ControlNetModel.from_pretrained(CHECKPOINT_PATH, torch_dtype=torch.float16)
pipe = StableDiffusionControlNetPipeline.from_pretrained(
    BASE_MODEL, controlnet=controlnet, torch_dtype=torch.float16, safety_checker=None
)
pipe.scheduler = UniPCMultistepScheduler.from_config(pipe.scheduler.config)
pipe.enable_model_cpu_offload()

# --- 3. HELPER: SAFE OVERLAY ---
def create_safe_overlay(generated_img, edge_map):
    gen_gray = cv2.cvtColor(np.array(generated_img), cv2.COLOR_RGB2GRAY)
    gen_gray = cv2.cvtColor(gen_gray, cv2.COLOR_GRAY2RGB)
    
    edge_data = np.array(edge_map)
    if len(edge_data.shape) == 3: edge_data = edge_data[:,:,0]
    
    # Resize to prevent crash
    edge_resized = cv2.resize(edge_data, (gen_gray.shape[1], gen_gray.shape[0]), interpolation=cv2.INTER_NEAREST)
    
    green_mask = np.zeros_like(gen_gray)
    green_mask[:, :, 1] = edge_resized
    return cv2.addWeighted(gen_gray, 0.5, green_mask, 1.0, 0)

# --- 4. PREPARE INPUT ---
# Pick random image
all_images = [os.path.join(r, f) for r, _, fs in os.walk(DATA_ROOT) for f in fs if f.lower().endswith(('.jpg', '.png'))]
random_path = random.choice(all_images)

original_img = Image.open(random_path).convert("RGB").resize((512, 512))
original_cv2 = np.array(original_img)

# Extract Edges
edges = cv2.Canny(original_cv2, 100, 200)
edges_rgb = np.stack([edges]*3, axis=2)
canny_pil = Image.fromarray(edges_rgb)

# --- 5. THE TRAFFIC LIGHT TEST ---
print("🚦 Running Color Steerability Test...")

prompts = [
    ("Structure Check", ""),  # Empty prompt
    # ("Red Paint", "a red car, automotive paint, highly detailed"),
    # ("Blue Paint", "a blue car, automotive paint, highly detailed"),
    # ("Yellow Paint", "a yellow car, automotive paint, highly detailed")
]

results = []
for title, prompt in prompts:
    print(f"   Generating: {title}...")
    # Using your "Strict Mode" settings
    out = pipe(
        prompt=prompt,
        negative_prompt="cartoon, drawing, sketch, blurry, low quality, grayscale",
        image=canny_pil,
        num_inference_steps=30,
        guidance_scale=7.5,        # High guidance to force the color
        controlnet_conditioning_scale=1.2 # Balanced control
    ).images[0]
    results.append((title, out))

# --- 6. VISUALIZE ---
fig, axs = plt.subplots(1, 5, figsize=(25, 5))

axs[0].imshow(canny_pil)
axs[0].set_title("Input Edges")
axs[0].axis("off")

for i, (title, img) in enumerate(results):
    axs[i+1].imshow(img)
    axs[i+1].set_title(title)
    axs[i+1].axis("off")

plt.show()

In [ ]:
import os
import cv2
import gc
import random
import numpy as np
import torch
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.cuda.amp import autocast, GradScaler
from diffusers import ControlNetModel, DDPMScheduler, AutoencoderKL, UNet2DConditionModel
from transformers import AutoTokenizer, CLIPTextModel
from tqdm.auto import tqdm
from scipy.spatial import KDTree

# --- 1. HARD RESET ---
torch.cuda.empty_cache()
gc.collect()

# --- 2. CONFIGURATION ---
MODEL_ID = "runwayml/stable-diffusion-v1-5"
OUTPUT_DIR = "/kaggle/working/controlnet_model_structure_fix"
DATA_PATH = "/kaggle/input/cars-colored/cars" 

NUM_EPOCHS = 5
BATCH_SIZE = 2  # Lower batch size for better stability
LEARNING_RATE = 1e-5
# CRITICAL CHANGE: 80% of training will have NO TEXT. 
# This forces the model to look at the edges.
EMPTY_PROMPT_RATE = 0.8 

if not os.path.exists(OUTPUT_DIR): os.makedirs(OUTPUT_DIR)

# --- 3. DATASET CLASS ---
class StrictStructureDataset(Dataset):
    def __init__(self, data_root, tokenizer, size=512, limit=1000):
        self.tokenizer = tokenizer
        self.size = size
        self.paths = []
        
        all_files = []
        for root, _, files in os.walk(data_root):
            for f in files:
                if f.lower().endswith(('.jpg', '.png', '.jpeg')):
                    all_files.append(os.path.join(root, f))
        all_files.sort()
        self.paths = all_files[:limit]
        print(f"📚 Training on {len(self.paths)} images (Structure Focused)")

        # Color Lookup (Still kept for the 20% of text training)
        self.color_map = {
            "black": (0,0,0), "white": (255,255,255), "red": (255,0,0),
            "green": (0,128,0), "blue": (0,0,255), "yellow": (255,255,0),
            "silver": (192,192,192), "gray": (128,128,128), "orange": (255,165,0),
            "purple": (128,0,128), "navy": (0,0,128), "gold": (255,215,0), "teal": (0,128,128)
        }
        self.color_names = list(self.color_map.keys())
        self.color_values = list(self.color_map.values())
        self.kdtree = KDTree(self.color_values)

    def get_color(self, img_bgr):
        small = cv2.resize(img_bgr, (64, 64))
        rgb = cv2.cvtColor(small, cv2.COLOR_BGR2RGB)
        h, w, _ = rgb.shape
        center = rgb[h//4:h*3//4, w//4:w*3//4]
        mean = center.mean(axis=(0,1))
        _, idx = self.kdtree.query(mean)
        return self.color_names[idx]

    def __len__(self): return len(self.paths)

    def __getitem__(self, idx):
        path = self.paths[idx]
        img_bgr = cv2.imread(path)
        if img_bgr is None: img_bgr = np.zeros((512, 512, 3), dtype=np.uint8)

        color_name = self.get_color(img_bgr)
        
        # Robust Canny (Randomized to prevent overfitting to one edge type)
        low = random.randint(50, 100)
        high = random.randint(150, 250)
        edges = cv2.Canny(img_bgr, low, high)
        edges_rgb = np.stack([edges]*3, axis=2)
        
        img_rgb = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2RGB)
        img_rgb = cv2.resize(img_rgb, (self.size, self.size))
        edges_rgb = cv2.resize(edges_rgb, (self.size, self.size))

        target = (img_rgb.astype(np.float32) / 127.5) - 1.0
        control = edges_rgb.astype(np.float32) / 255.0

        # SMART PROMPTING 2.0
        # 80% Empty: Forces "Look at the edges!"
        # 20% Color: Forces "Look at the paint!"
        if random.random() < EMPTY_PROMPT_RATE:
            prompt = "" 
        else:
            prompt = f"a {color_name} car, automotive paint"

        inputs = self.tokenizer(prompt, max_length=77, padding="max_length", truncation=True, return_tensors="pt")

        return {
            "pixel_values": torch.from_numpy(target).permute(2,0,1),
            "control_values": torch.from_numpy(control).permute(2,0,1),
            "input_ids": inputs.input_ids[0]
        }

# --- 4. MODEL LOADING ---
print("⏳ Loading Pipeline...")
tokenizer = AutoTokenizer.from_pretrained(MODEL_ID, subfolder="tokenizer")
noise_scheduler = DDPMScheduler.from_pretrained(MODEL_ID, subfolder="scheduler")
text_encoder = CLIPTextModel.from_pretrained(MODEL_ID, subfolder="text_encoder")
vae = AutoencoderKL.from_pretrained(MODEL_ID, subfolder="vae")
unet = UNet2DConditionModel.from_pretrained(MODEL_ID, subfolder="unet")

controlnet = ControlNetModel.from_unet(unet)

vae.requires_grad_(False); unet.requires_grad_(False); text_encoder.requires_grad_(False)
vae.to(DEVICE, dtype=torch.float16)
unet.to(DEVICE, dtype=torch.float16)
text_encoder.to(DEVICE, dtype=torch.float16)
controlnet.to(DEVICE, dtype=torch.float32) 
controlnet.train()
controlnet.enable_gradient_checkpointing()

# --- 5. EXECUTE ---
optimizer = torch.optim.AdamW(controlnet.parameters(), lr=LEARNING_RATE)
dataset = StrictStructureDataset(DATA_PATH, tokenizer, limit=1000)

dataloader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=2)
scaler = GradScaler()
print(f"🚀 Starting Structure-First Training...")

for epoch in range(NUM_EPOCHS):
    pbar = tqdm(dataloader, desc=f"Epoch {epoch+1}/{NUM_EPOCHS}")
    for batch in pbar:
        optimizer.zero_grad()
        
        pixel_values = batch["pixel_values"].to(DEVICE, dtype=torch.float16)
        control_values = batch["control_values"].to(DEVICE, dtype=torch.float16)
        input_ids = batch["input_ids"].to(DEVICE)

        with autocast(dtype=torch.float16):
            latents = vae.encode(pixel_values).latent_dist.sample() * vae.config.scaling_factor
            encoder_hidden_states = text_encoder(input_ids)[0]
            noise = torch.randn_like(latents)
            timesteps = torch.randint(0, noise_scheduler.config.num_train_timesteps, (latents.shape[0],), device=DEVICE).long()
            noisy_latents = noise_scheduler.add_noise(latents, noise, timesteps)

            down_res, mid_res = controlnet(
                noisy_latents,
                timesteps,
                encoder_hidden_states=encoder_hidden_states,
                controlnet_cond=control_values,
                return_dict=False
            )

            noise_pred = unet(
                noisy_latents,
                timesteps,
                encoder_hidden_states=encoder_hidden_states,
                down_block_additional_residuals=down_res,
                mid_block_additional_residual=mid_res
            ).sample

            loss = F.mse_loss(noise_pred.float(), noise.float(), reduction="mean")

        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        pbar.set_postfix(loss=loss.item())

    save_path = os.path.join(OUTPUT_DIR, f"checkpoint-epoch-{epoch+1}")
    controlnet.save_pretrained(save_path)
    print(f"💾 Saved: {save_path}")
print("✅ Training Complete.")

In [ ]:
# Point to Epoch 3 (The last fully completed epoch)
CHECKPOINT_PATH = "/kaggle/working/controlnet_model_final_1k/checkpoint-epoch-3"

In [ ]:
import os
import cv2
import torch
import random
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from diffusers import StableDiffusionControlNetPipeline, ControlNetModel, UniPCMultistepScheduler
from scipy.spatial import KDTree

# --- 1. CONFIGURATION ---
# ✅ CORRECTED PATH: Pointing to Epoch 3
CHECKPOINT_PATH = "/kaggle/working/controlnet_model_final_1k/checkpoint-epoch-3"
BASE_MODEL = "runwayml/stable-diffusion-v1-5"
DATA_PATH = "/kaggle/input/cars-colored/cars"

# Safety Check: Does the file exist?
if not os.path.exists(CHECKPOINT_PATH):
    print(f"⚠️ WARNING: Could not find {CHECKPOINT_PATH}")
    print("Checking available checkpoints...")
    saved_models = sorted(os.listdir("/kaggle/working/controlnet_model_final_1k"))
    print(f"Found: {saved_models}")
    # Update path to the last one found
    if saved_models:
        CHECKPOINT_PATH = os.path.join("/kaggle/working/controlnet_model_final_1k", saved_models[-1])
        print(f"🔄 Auto-switching to: {CHECKPOINT_PATH}")

# --- 2. LOAD PIPELINE ---
print(f"⏳ Loading Model from: {CHECKPOINT_PATH}")
controlnet = ControlNetModel.from_pretrained(CHECKPOINT_PATH, torch_dtype=torch.float16)
pipe = StableDiffusionControlNetPipeline.from_pretrained(
    BASE_MODEL, controlnet=controlnet, torch_dtype=torch.float16, safety_checker=None
)
pipe.scheduler = UniPCMultistepScheduler.from_config(pipe.scheduler.config)
pipe.enable_model_cpu_offload()

# --- 3. HELPER: COLOR EXTRACTOR ---
def get_dominant_color(image_cv2):
    color_map = {
        "black": (0,0,0), "white": (255,255,255), "red": (255,0,0),
        "green": (0,128,0), "blue": (0,0,255), "yellow": (255,255,0),
        "silver": (192,192,192), "gray": (128,128,128), "orange": (255,165,0),
        "purple": (128,0,128), "navy": (0,0,128), "gold": (255,215,0)
    }
    names = list(color_map.keys())
    values = list(color_map.values())
    tree = KDTree(values)
    
    small = cv2.resize(image_cv2, (64, 64))
    rgb = cv2.cvtColor(small, cv2.COLOR_BGR2RGB)
    h, w, _ = rgb.shape
    center = rgb[h//4:h*3//4, w//4:w*3//4]
    mean = center.mean(axis=(0,1))
    _, idx = tree.query(mean)
    return names[idx]

# --- 4. PREPARE INPUT ---
all_images = [os.path.join(r, f) for r, _, fs in os.walk(DATA_PATH) for f in fs if f.lower().endswith(('.jpg', '.png'))]
if not all_images:
    raise ValueError("No images found in data path! Check your path.")

random_path = random.choice(all_images)

# Load and process
original_bgr = cv2.imread(random_path)
original_rgb = cv2.cvtColor(original_bgr, cv2.COLOR_BGR2RGB)
detected_color = get_dominant_color(original_bgr)

# Generate Edges
low = random.randint(50, 100)
high = random.randint(150, 250)
edges = cv2.Canny(original_bgr, low, high)
edges_rgb = np.stack([edges]*3, axis=2)

original_pil = Image.fromarray(original_rgb).resize((512, 512))
canny_pil = Image.fromarray(edges_rgb).resize((512, 512))

print(f"🚗 Selected Image: {os.path.basename(random_path)}")
print(f"🎨 Detected Paint: {detected_color.upper()}")

# --- 5. RUN INFERENCE ---
print("🧪 Generating...")

# A. STRUCTURE CHECK (Empty Prompt)
struct_out = pipe(
    prompt="", 
    negative_prompt="blurry, distorted, low quality, artifact",
    image=canny_pil,
    num_inference_steps=30,
    guidance_scale=4.0, 
).images[0]

# B. FINAL GENERATION (Color Prompt)
final_out = pipe(
    prompt=f"a {detected_color} car, automotive paint, highly detailed, realistic",
    negative_prompt="cartoon, sketch, wireframe, drawing, blurry, low quality",
    image=canny_pil,
    num_inference_steps=30,
    guidance_scale=7.5,
).images[0]

# --- 6. VISUALIZE ---
fig, axs = plt.subplots(1, 4, figsize=(24, 6))

axs[0].imshow(original_pil)
axs[0].set_title("Ground Truth")
axs[0].axis("off")

axs[1].imshow(canny_pil)
axs[1].set_title("Input Edges")
axs[1].axis("off")

axs[2].imshow(struct_out)
axs[2].set_title("Structure Check\n(Prompt: \"\")")
axs[2].axis("off")

axs[3].imshow(final_out)
axs[3].set_title(f"Final Generation\n(Prompt: \"{detected_color} car\")")
axs[3].axis("off")

plt.show()

In [ ]:
import torch
from diffusers import StableDiffusionControlNetPipeline, ControlNetModel, UniPCMultistepScheduler

# 1. Load your "Good Structure" Model (Epoch 3 or 5 from previous run)
# (Ensure this points to the model that successfully drew the truck/lamborghini shape)
CHECKPOINT_PATH = "/kaggle/working/controlnet_model_final_1k/checkpoint-epoch-3" 
BASE_MODEL = "runwayml/stable-diffusion-v1-5"

# 2. Load Pipeline with IP-Adapter
print("⏳ Loading Hybrid Pipeline...")
controlnet = ControlNetModel.from_pretrained(CHECKPOINT_PATH, torch_dtype=torch.float16)
pipe = StableDiffusionControlNetPipeline.from_pretrained(
    BASE_MODEL, controlnet=controlnet, torch_dtype=torch.float16, safety_checker=None
)

# 3. Load the IP-Adapter (The "Style" Plugin)
# We load the weights directly from HuggingFace
pipe.load_ip_adapter("h94/IP-Adapter", subfolder="models", weight_name="ip-adapter_sd15.bin")

# Optimized Scheduler
pipe.scheduler = UniPCMultistepScheduler.from_config(pipe.scheduler.config)
pipe.enable_model_cpu_offload()

print("✅ Hybrid Pipeline Ready: ControlNet (Shape) + IP-Adapter (Color)")

In [ ]:
import io
import cv2
import numpy as np
import ipywidgets as widgets
from IPython.display import display, clear_output
from PIL import Image
import matplotlib.pyplot as plt

# --- CONFIGURATION ---
# Scale: 0.0 (No Style) to 1.0 (Full Style Copy)
STYLE_STRENGTH = 0.6    
# Control: 0.0 (No Shape) to 2.0 (Rigid Shape)
SHAPE_STRENGTH = 1.5    

def decode_image(uploader_value):
    if isinstance(uploader_value, tuple): content = uploader_value[0]['content']
    else: key = list(uploader_value.keys())[0]; content = uploader_value[key]['content']
    return Image.open(io.BytesIO(content)).convert("RGB")

# --- UI ---
btn_struct = widgets.FileUpload(accept='.jpg,.png, .jpeg', description="1. Shape Ref")
btn_style = widgets.FileUpload(accept='.jpg,.png, .jpeg', description="2. Style Ref (Starry Night)")
btn_run = widgets.Button(description="🚀 Hybrid Generate", button_style='primary')
out = widgets.Output()

def run_hybrid(b):
    with out:
        clear_output()
        if not btn_struct.value or not btn_style.value: print("⚠️ Upload both images."); return
        
        try:
            print("Processing...")
            img_struct = decode_image(btn_struct.value).resize((512, 512))
            img_style = decode_image(btn_style.value).resize((512, 512))

            # 1. Prepare Edges
            img_cv = np.array(img_struct)
            edges = cv2.Canny(img_cv, 100, 200)
            edges_rgb = np.stack([edges]*3, axis=2)
            canny_pil = Image.fromarray(edges_rgb)

            # 2. Configure Adapter Strength
            pipe.set_ip_adapter_scale(STYLE_STRENGTH)

            # 3. Generate
            # Notice the prompt is generic ("a car"). 
            # We let the images do the talking.
            result = pipe(
                prompt="high quality, masterpiece, 8k uhd", 
                negative_prompt="blurry, low quality, distorted, wireframe, sketch",
                image=canny_pil,                  # ControlNet Input
                ip_adapter_image=img_style,       # IP-Adapter Input
                num_inference_steps=30,
                guidance_scale=5.0,
                controlnet_conditioning_scale=SHAPE_STRENGTH
            ).images[0]

            # 4. Show
            fig, axs = plt.subplots(1, 3, figsize=(18, 6))
            axs[0].imshow(canny_pil); axs[0].set_title("Input Shape (ControlNet)"); axs[0].axis("off")
            axs[1].imshow(img_style); axs[1].set_title("Input Style (IP-Adapter)"); axs[1].axis("off")
            axs[2].imshow(result); axs[2].set_title("Result"); axs[2].axis("off")
            plt.show()

        except Exception as e: print(f"Error: {e}")

btn_run.on_click(run_hybrid)
display(widgets.HBox([btn_struct, btn_style]))
display(btn_run)
display(out)

In [ ]:
import io
import cv2
import numpy as np
import ipywidgets as widgets
from IPython.display import display, clear_output
from PIL import Image
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import torch
from diffusers import StableDiffusionControlNetPipeline, ControlNetModel, UniPCMultistepScheduler

# --- 1. CONFIGURATION ---
# We use your "Structure-First" model (Epoch 3) because it respects geometry best
CHECKPOINT_PATH = "/kaggle/working/controlnet_model_final_1k/checkpoint-epoch-3" 
BASE_MODEL = "runwayml/stable-diffusion-v1-5"

# Tuning for "Color Only"
GUIDANCE_SCALE = 7.5       # High guidance to force the exact colors
CONTROL_SCALE = 1.0        # Balanced structure (not too rigid, not too loose)
INFERENCE_STEPS = 40

# --- 2. LOAD PIPELINE ---
print("⏳ Loading ControlNet Pipeline...")
controlnet = ControlNetModel.from_pretrained(CHECKPOINT_PATH, torch_dtype=torch.float16)
pipe = StableDiffusionControlNetPipeline.from_pretrained(
    BASE_MODEL, controlnet=controlnet, torch_dtype=torch.float16, safety_checker=None
)
pipe.scheduler = UniPCMultistepScheduler.from_config(pipe.scheduler.config)
pipe.enable_model_cpu_offload()

# --- 3. HELPER: ROBUST PALETTE EXTRACTOR ---
def get_detailed_palette(image_rgb, k=3):
    # 1. Resize for speed
    img_small = cv2.resize(image_rgb, (64, 64))
    pixels = img_small.reshape(-1, 3)
    
    # 2. Extract dominant clusters
    kmeans = KMeans(n_clusters=k, n_init=10).fit(pixels)
    colors = kmeans.cluster_centers_.astype(int)
    
    # 3. Smart Namer (Maps RGB to descriptive SD-friendly names)
    # We use a larger dictionary to catch nuances like "Midnight Blue" or "Gold"
    color_map = {
        "midnight blue": (25, 25, 112), "navy": (0, 0, 128), "royal blue": (65, 105, 225),
        "gold": (255, 215, 0), "yellow": (255, 255, 0), "amber": (255, 191, 0),
        "crimson": (220, 20, 60), "red": (255, 0, 0), "maroon": (128, 0, 0),
        "emerald": (80, 200, 120), "green": (0, 128, 0), "olive": (128, 128, 0),
        "silver": (192, 192, 192), "gray": (128, 128, 128), "white": (255, 255, 255),
        "black": (10, 10, 10), "charcoal": (54, 69, 79), "beige": (245, 245, 220)
    }
    
    def get_name(rgb_val):
        min_dist = float('inf')
        best_name = "gray"
        for name, val in color_map.items():
            dist = np.linalg.norm(np.array(rgb_val) - np.array(val))
            if dist < min_dist:
                min_dist = dist
                best_name = name
        return best_name

    # Sort by frequency (Darker colors often body, lighter often trim/lights)
    unique, counts = np.unique(kmeans.labels_, return_counts=True)
    sorted_indices = np.argsort(counts)[::-1]
    sorted_colors = colors[sorted_indices]
    
    return sorted_colors, [get_name(c) for c in sorted_colors]

def decode_image(uploader_value):
    if isinstance(uploader_value, tuple): content = uploader_value[0]['content']
    else: key = list(uploader_value.keys())[0]; content = uploader_value[key]['content']
    return np.array(Image.open(io.BytesIO(content)).convert("RGB"))

# --- 4. THE UI ---
btn_struct = widgets.FileUpload(accept='.jpg,.png, .jpeg', description="1. Shape Ref")
btn_color = widgets.FileUpload(accept='.jpg,.png, .jpeg', description="2. Color Palette Ref")
btn_run = widgets.Button(description="🎨 Inject Palette", button_style='success', layout=widgets.Layout(width='300px'))
out = widgets.Output()

def run_palette_injection(b):
    with out:
        clear_output()
        if not btn_struct.value or not btn_color.value: print("⚠️ Upload both images."); return
        
        try:
            print("⏳ Analyzing Palette...")
            img_struct = decode_image(btn_struct.value)
            img_color = decode_image(btn_color.value)

            # 1. Prepare Shape
            img_struct_512 = cv2.resize(img_struct, (512, 512))
            edges = cv2.Canny(img_struct_512, 100, 200)
            edges_rgb = np.stack([edges]*3, axis=2)
            canny_pil = Image.fromarray(edges_rgb)

            # 2. Extract Palette
            palette_rgb, palette_names = get_detailed_palette(img_color, k=3)
            primary = palette_names[0]
            secondary = palette_names[1]
            accent = palette_names[2]
            
            print(f"🎨 Detected Palette: {primary.upper()} (Body) + {secondary.upper()} (Trim)")

            # 3. Construct "Anti-Style" Prompt
            # We explicitly describe the MATERIALS to force photorealism
            prompt = (
                f"a photorealistic car, automotive gloss paint finish in {primary}, "
                f"with {secondary} accents and {accent} details, "
                f"professional car photography, 8k, highly detailed, sharp focus, ray tracing"
            )
            
            # 4. Generate
            result = pipe(
                prompt,
                # Negative prompt forbids the "Starry Night" STYLE while keeping the COLOR
                negative_prompt="impressionism, van gogh, style, painting, brushstrokes, swirl, drawing, sketch, cartoon, messy, blurry",
                image=canny_pil,
                num_inference_steps=INFERENCE_STEPS,
                guidance_scale=GUIDANCE_SCALE,
                controlnet_conditioning_scale=CONTROL_SCALE
            ).images[0]

            # 5. Visualize
            fig, axs = plt.subplots(1, 3, figsize=(18, 6))
            
            axs[0].imshow(img_struct_512)
            axs[0].set_title("Input Shape")
            axs[0].axis("off")
            
            # Show Palette Bar below Color Ref
            p_bar = np.zeros((50, 512, 3), dtype=np.uint8)
            step = 512//3
            for i in range(3): p_bar[:, i*step:(i+1)*step] = palette_rgb[i]
            
            # Combine Color Ref + Palette Bar
            combined_color_view = np.vstack([cv2.resize(img_color, (512, 462)), p_bar])
            axs[1].imshow(combined_color_view)
            axs[1].set_title(f"Palette: {primary}/{secondary}/{accent}")
            axs[1].axis("off")
            
            axs[2].imshow(result)
            axs[2].set_title("Result: Shape + Palette Only")
            axs[2].axis("off")
            plt.show()

        except Exception as e: print(f"Error: {e}")

btn_run.on_click(run_palette_injection)
display(widgets.HBox([btn_struct, btn_color]))
display(btn_run)
display(out)

In [ ]:
import io
import os
import cv2
import torch
import random
import numpy as np
import ipywidgets as widgets
from IPython.display import display, clear_output
from PIL import Image
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
from diffusers import StableDiffusionControlNetPipeline, ControlNetModel, UniPCMultistepScheduler

# --- 1. CONFIGURATION ---
CHECKPOINT_PATH = "/kaggle/working/controlnet_model_final_1k/checkpoint-epoch-3" 
BASE_MODEL = "runwayml/stable-diffusion-v1-5"
DATA_PATH = "/kaggle/input/cars-colored/cars" # Your training data

# "Strict Palette" Settings
GUIDANCE_SCALE = 7.5
CONTROL_SCALE = 1.0
INFERENCE_STEPS = 30 # Slightly lower for faster batch processing

# --- 2. LOAD PIPELINE ---
print("⏳ Loading Pipeline...")
controlnet = ControlNetModel.from_pretrained(CHECKPOINT_PATH, torch_dtype=torch.float16)
pipe = StableDiffusionControlNetPipeline.from_pretrained(
    BASE_MODEL, controlnet=controlnet, torch_dtype=torch.float16, safety_checker=None
)
pipe.scheduler = UniPCMultistepScheduler.from_config(pipe.scheduler.config)
pipe.enable_model_cpu_offload()

# --- 3. HELPER FUNCTIONS ---
def get_detailed_palette(image_rgb, k=3):
    img_small = cv2.resize(image_rgb, (64, 64))
    pixels = img_small.reshape(-1, 3)
    kmeans = KMeans(n_clusters=k, n_init=10).fit(pixels)
    colors = kmeans.cluster_centers_.astype(int)
    
    # Extended Color Map
    color_map = {
        "midnight blue": (25, 25, 112), "navy": (0, 0, 128), "royal blue": (65, 105, 225),
        "gold": (255, 215, 0), "yellow": (255, 255, 0), "amber": (255, 191, 0),
        "crimson": (220, 20, 60), "red": (255, 0, 0), "maroon": (128, 0, 0),
        "emerald": (80, 200, 120), "green": (0, 128, 0), "olive": (128, 128, 0),
        "silver": (192, 192, 192), "gray": (128, 128, 128), "white": (255, 255, 255),
        "black": (10, 10, 10), "charcoal": (54, 69, 79), "beige": (245, 245, 220),
        "teal": (0, 128, 128), "orange": (255, 165, 0), "purple": (128, 0, 128)
    }
    
    def get_name(rgb_val):
        min_dist = float('inf'); best_name = "gray"
        for name, val in color_map.items():
            dist = np.linalg.norm(np.array(rgb_val) - np.array(val))
            if dist < min_dist: min_dist = dist; best_name = name
        return best_name

    unique, counts = np.unique(kmeans.labels_, return_counts=True)
    sorted_indices = np.argsort(counts)[::-1]
    sorted_colors = colors[sorted_indices]
    
    return sorted_colors, [get_name(c) for c in sorted_colors]

def decode_image(uploader_value):
    if isinstance(uploader_value, tuple): content = uploader_value[0]['content']
    else: key = list(uploader_value.keys())[0]; content = uploader_value[key]['content']
    return np.array(Image.open(io.BytesIO(content)).convert("RGB"))

# --- 4. BATCH LOGIC ---
btn_style = widgets.FileUpload(accept='.jpg,.png, .jpeg', description="Upload Style Ref")
btn_batch = widgets.Button(description="🚀 Run Batch (10 Images)", button_style='warning', layout=widgets.Layout(width='300px'))
out = widgets.Output()

def run_batch(b):
    with out:
        clear_output()
        if not btn_style.value: print("⚠️ Please upload a Style Reference image first."); return
        
        try:
            # A. Analyze Style
            print("🎨 Analyzing Style Reference...")
            img_style = decode_image(btn_style.value)
            palette_rgb, palette_names = get_detailed_palette(img_style, k=3)
            p1, p2, p3 = palette_names[0], palette_names[1], palette_names[2]
            print(f"   Detected Palette: {p1.upper()} (Main) / {p2.upper()} (Sec) / {p3.upper()} (Acc)")
            
            # B. Pick 10 Random Images
            all_files = [os.path.join(r, f) for r, _, fs in os.walk(DATA_PATH) for f in fs if f.lower().endswith(('.jpg', '.png'))]
            if len(all_files) < 10: selected_files = all_files
            else: selected_files = random.sample(all_files, 10)
            
            print(f"📸 Selected {len(selected_files)} random cars from training set...")

            # C. Generate Grid
            fig, axs = plt.subplots(len(selected_files), 3, figsize=(15, 4 * len(selected_files)))
            
            for i, path in enumerate(selected_files):
                # 1. Load & Edge Detect
                orig_bgr = cv2.imread(path)
                orig_rgb = cv2.cvtColor(orig_bgr, cv2.COLOR_BGR2RGB)
                orig_512 = cv2.resize(orig_rgb, (512, 512))
                
                low = random.randint(50, 100); high = random.randint(150, 250)
                edges = cv2.Canny(orig_512, low, high)
                edges_rgb = np.stack([edges]*3, axis=2)
                canny_pil = Image.fromarray(edges_rgb)
                
                # 2. Construct Prompt
                prompt = (
                    f"a photorealistic car, automotive gloss paint finish in {p1}, "
                    f"with {p2} accents and {p3} details, "
                    f"professional car photography, 8k, highly detailed, sharp focus"
                )

                # 3. Generate
                print(f"   [{i+1}/10] Generating {os.path.basename(path)}...")
                result = pipe(
                    prompt,
                    negative_prompt="impressionism, painting, drawing, sketch, cartoon, messy, blurry, low quality",
                    image=canny_pil,
                    num_inference_steps=INFERENCE_STEPS,
                    guidance_scale=GUIDANCE_SCALE,
                    controlnet_conditioning_scale=CONTROL_SCALE
                ).images[0]
                
                # 4. Plot Row
                # Col 1: Original Structure
                axs[i, 0].imshow(orig_512)
                axs[i, 0].set_title(f"Original: {os.path.basename(path)}")
                axs[i, 0].axis("off")
                
                # Col 2: Style Ref (Small inset)
                axs[i, 1].imshow(img_style)
                axs[i, 1].set_title(f"Target: {p1}/{p2}")
                axs[i, 1].axis("off")
                
                # Col 3: Result
                axs[i, 2].imshow(result)
                axs[i, 2].set_title("Resprayed Result")
                axs[i, 2].axis("off")
            
            plt.tight_layout()
            plt.show()
            print("✅ Batch Complete.")

        except Exception as e: print(f"❌ Error: {e}")

btn_batch.on_click(run_batch)
display(btn_style, btn_batch, out)

In [ ]:
import io
import os
import cv2
import torch
import random
import numpy as np
import ipywidgets as widgets
from IPython.display import display, clear_output
from PIL import Image
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
from diffusers import StableDiffusionControlNetPipeline, ControlNetModel, UniPCMultistepScheduler

# --- 1. CONFIGURATION ---
CHECKPOINT_PATH = "/kaggle/working/controlnet_model_final_1k/checkpoint-epoch-3" 
BASE_MODEL = "runwayml/stable-diffusion-v1-5"
DATA_PATH = "/kaggle/input/cars-colored/cars" 

GUIDANCE_SCALE = 7.5
CONTROL_SCALE = 1.0
INFERENCE_STEPS = 30 

# --- 2. LOAD PIPELINE ---
# (Only load if not already loaded to save time)
if 'pipe' not in globals():
    print("⏳ Loading Pipeline...")
    controlnet = ControlNetModel.from_pretrained(CHECKPOINT_PATH, torch_dtype=torch.float16)
    pipe = StableDiffusionControlNetPipeline.from_pretrained(
        BASE_MODEL, controlnet=controlnet, torch_dtype=torch.float16, safety_checker=None
    )
    pipe.scheduler = UniPCMultistepScheduler.from_config(pipe.scheduler.config)
    pipe.enable_model_cpu_offload()

# --- 3. HELPER FUNCTIONS ---
def get_detailed_palette(image_rgb, k=3):
    img_small = cv2.resize(image_rgb, (64, 64))
    pixels = img_small.reshape(-1, 3)
    kmeans = KMeans(n_clusters=k, n_init=10).fit(pixels)
    colors = kmeans.cluster_centers_.astype(int)
    
    color_map = {
        "midnight blue": (25, 25, 112), "navy": (0, 0, 128), "royal blue": (65, 105, 225),
        "gold": (255, 215, 0), "yellow": (255, 255, 0), "amber": (255, 191, 0),
        "crimson": (220, 20, 60), "red": (255, 0, 0), "maroon": (128, 0, 0),
        "emerald": (80, 200, 120), "green": (0, 128, 0), "olive": (128, 128, 0),
        "silver": (192, 192, 192), "gray": (128, 128, 128), "white": (255, 255, 255),
        "black": (10, 10, 10), "charcoal": (54, 69, 79), "beige": (245, 245, 220),
        "teal": (0, 128, 128), "orange": (255, 165, 0), "purple": (128, 0, 128)
    }
    
    def get_name(rgb_val):
        min_dist = float('inf'); best_name = "gray"
        for name, val in color_map.items():
            dist = np.linalg.norm(np.array(rgb_val) - np.array(val))
            if dist < min_dist: min_dist = dist; best_name = name
        return best_name

    unique, counts = np.unique(kmeans.labels_, return_counts=True)
    sorted_indices = np.argsort(counts)[::-1]
    sorted_colors = colors[sorted_indices]
    return sorted_colors, [get_name(c) for c in sorted_colors]

def decode_image(uploader_value):
    if isinstance(uploader_value, tuple): content = uploader_value[0]['content']
    else: key = list(uploader_value.keys())[0]; content = uploader_value[key]['content']
    return np.array(Image.open(io.BytesIO(content)).convert("RGB"))

# --- 4. BATCH LOGIC ---
btn_style = widgets.FileUpload(accept='.jpg,.png,.jpeg', description="Upload Style Ref")
btn_batch = widgets.Button(description="🚀 Run Batch (10 Images)", button_style='warning', layout=widgets.Layout(width='300px'))
out = widgets.Output()

def run_batch(b):
    # Clear only ONCE at the start
    out.clear_output()
    
    with out:
        if not btn_style.value: 
            print("⚠️ Please upload a Style Reference image first.")
            return
        
        try:
            # A. Analyze Style
            print("🎨 Analyzing Style Reference...")
            img_style = decode_image(btn_style.value)
            palette_rgb, palette_names = get_detailed_palette(img_style, k=3)
            p1, p2, p3 = palette_names[0], palette_names[1], palette_names[2]
            print(f"   Detected Palette: {p1.upper()} (Main) / {p2.upper()} (Sec) / {p3.upper()} (Acc)")
            
            # B. Pick 10 Random Images
            all_files = [os.path.join(r, f) for r, _, fs in os.walk(DATA_PATH) for f in fs if f.lower().endswith(('.jpg', '.png'))]
            if len(all_files) < 10: selected_files = all_files
            else: selected_files = random.sample(all_files, 10)
            
            print(f"📸 Selected {len(selected_files)} random cars. Generating...")

            # C. Loop & Generate (Plotting separately to avoid memory issues/glitches)
            for i, path in enumerate(selected_files):
                # 1. Load & Edge Detect
                orig_bgr = cv2.imread(path)
                orig_rgb = cv2.cvtColor(orig_bgr, cv2.COLOR_BGR2RGB)
                orig_512 = cv2.resize(orig_rgb, (512, 512))
                
                low = random.randint(50, 100); high = random.randint(150, 250)
                edges = cv2.Canny(orig_512, low, high)
                edges_rgb = np.stack([edges]*3, axis=2)
                canny_pil = Image.fromarray(edges_rgb)
                
                # 2. Prompt
                prompt = (
                    f"a photorealistic car, automotive gloss paint finish in {p1}, "
                    f"with {p2} accents and {p3} details, "
                )

                # 3. Generate
                print(f"   ⏳ Generating {i+1}/{len(selected_files)}: {os.path.basename(path)}...")
                result = pipe(
                    prompt,
                    negative_prompt="impressionism, painting, drawing, sketch, cartoon, messy, blurry, low quality",
                    image=canny_pil,
                    num_inference_steps=INFERENCE_STEPS,
                    guidance_scale=GUIDANCE_SCALE,
                    controlnet_conditioning_scale=CONTROL_SCALE
                ).images[0]
                
                # 4. Create Individual Figure for each (Safer for Widgets)
                fig, axs = plt.subplots(1, 3, figsize=(18, 5))
                
                axs[0].imshow(orig_512)
                axs[0].set_title(f"Original: {os.path.basename(path)}")
                axs[0].axis("off")
                
                axs[1].imshow(img_style)
                axs[1].set_title(f"Target: {p1}/{p2}")
                axs[1].axis("off")
                
                axs[2].imshow(result)
                axs[2].set_title("Result")
                axs[2].axis("off")
                
                # Use display() explicitly
                display(fig)
                plt.close(fig) # Close to save memory

            print("✅ Batch Complete.")

        except Exception as e: print(f"❌ Error: {e}")

btn_batch.on_click(run_batch)
display(widgets.HBox([btn_style, btn_batch]))
display(out)

In [ ]:
import io
import os
import cv2
import torch
import random
import numpy as np
import ipywidgets as widgets
from IPython.display import display, clear_output
from PIL import Image
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
from diffusers import StableDiffusionControlNetPipeline, ControlNetModel, UniPCMultistepScheduler
from scipy.spatial import KDTree

# --- 1. CONFIGURATION ---
CHECKPOINT_PATH = "/kaggle/working/controlnet_model_final_1k/checkpoint-epoch-3"
BASE_MODEL = "runwayml/stable-diffusion-v1-5"
DATA_PATH = "/kaggle/input/cars-colored/cars"

# Inference Settings (Standard)
GUIDANCE_SCALE = 7.5
CONTROL_SCALE = 1.0
INFERENCE_STEPS = 30

# --- 2. LOAD PIPELINE ---
if 'pipe' not in globals():
    print("⏳ Loading Pipeline...")
    controlnet = ControlNetModel.from_pretrained(CHECKPOINT_PATH, torch_dtype=torch.float16)
    pipe = StableDiffusionControlNetPipeline.from_pretrained(
        BASE_MODEL, controlnet=controlnet, torch_dtype=torch.float16, safety_checker=None
    )
    pipe.scheduler = UniPCMultistepScheduler.from_config(pipe.scheduler.config)
    pipe.enable_model_cpu_offload()

# --- 3. HELPER: TRAIN-TIME COLOR EXTRACTOR ---
# We use the exact same logic used in your training loop to ensure consistency
def get_training_color(image_bgr):
    # Standard color map used in training
    color_map = {
        "black": (0,0,0), "white": (255,255,255), "red": (255,0,0),
        "green": (0,128,0), "blue": (0,0,255), "yellow": (255,255,0),
        "silver": (192,192,192), "gray": (128,128,128), "orange": (255,165,0),
        "purple": (128,0,128), "navy": (0,0,128), "gold": (255,215,0), "teal": (0,128,128)
    }
    names = list(color_map.keys())
    values = list(color_map.values())
    tree = KDTree(values)
    
    small = cv2.resize(image_bgr, (64, 64))
    rgb = cv2.cvtColor(small, cv2.COLOR_BGR2RGB)
    
    # Center crop logic (same as training)
    h, w, _ = rgb.shape
    center = rgb[h//4:h*3//4, w//4:w*3//4]
    mean = center.mean(axis=(0,1))
    
    _, idx = tree.query(mean)
    return names[idx]

# --- 4. RECONSTRUCTION LOOP ---
btn_test = widgets.Button(description="🔄 Test Training Recall", button_style='success', layout=widgets.Layout(width='300px'))
out = widgets.Output()

def run_reconstruction_test(b):
    out.clear_output()
    with out:
        # 1. Pick 10 Random Images from Training Set
        all_files = [os.path.join(r, f) for r, _, fs in os.walk(DATA_PATH) for f in fs if f.lower().endswith(('.jpg', '.png'))]
        selected_files = random.sample(all_files, 10)
        
        print(f"🧪 Testing on {len(selected_files)} training images...")
        
        for i, path in enumerate(selected_files):
            # A. Load Original
            orig_bgr = cv2.imread(path)
            orig_rgb = cv2.cvtColor(orig_bgr, cv2.COLOR_BGR2RGB)
            orig_512 = cv2.resize(orig_rgb, (512, 512))
            
            # B. Recreate Training Inputs
            # 1. Extract Color Label (Prompt)
            detected_color = get_training_color(orig_bgr)
            
            # 2. Extract Edges (Control)
            # Using random thresholds like training to see if it generalizes
            low = random.randint(50, 100); high = random.randint(150, 250)
            edges = cv2.Canny(orig_512, low, high)
            edges_rgb = np.stack([edges]*3, axis=2)
            canny_pil = Image.fromarray(edges_rgb)
            
            # C. Generate
            prompt = f"a {detected_color} car, automotive paint"
            print(f"   [{i+1}/10] Generating: '{prompt}'...")
            
            result = pipe(
                prompt,
                negative_prompt="blurry, low quality, sketch, cartoon",
                image=canny_pil,
                num_inference_steps=INFERENCE_STEPS,
                guidance_scale=GUIDANCE_SCALE,
                controlnet_conditioning_scale=CONTROL_SCALE
            ).images[0]
            
            # D. Visualize Comparison
            fig, axs = plt.subplots(1, 3, figsize=(18, 5))
            
            # 1. Ground Truth (What the model saw during training)
            axs[0].imshow(orig_512)
            axs[0].set_title(f"Ground Truth ({detected_color})")
            axs[0].axis("off")
            
            # 2. Input Edges (The hint it got)
            axs[1].imshow(canny_pil)
            axs[1].set_title("Input Edges")
            axs[1].axis("off")
            
            # 3. Model Output (What it remembers)
            axs[2].imshow(result)
            axs[2].set_title("Reconstruction")
            axs[2].axis("off")
            
            display(fig)
            plt.close(fig)

        print("✅ Test Complete.")

btn_test.on_click(run_reconstruction_test)
display(btn_test, out)

In [ ]:
import os
import cv2
import gc
import random
import numpy as np
import torch
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.cuda.amp import autocast, GradScaler
from diffusers import ControlNetModel, DDPMScheduler, AutoencoderKL, UNet2DConditionModel, StableDiffusionControlNetPipeline
from transformers import AutoTokenizer, CLIPTextModel
from tqdm.auto import tqdm
from scipy.spatial import KDTree
import matplotlib.pyplot as plt
from PIL import Image

# --- 1. MEMORY & CONFIGURATION ---
torch.cuda.empty_cache()
gc.collect()

MODEL_ID = "runwayml/stable-diffusion-v1-5"
OUTPUT_DIR = "/kaggle/working/controlnet_model_balanced"
VALIDATION_DIR = "/kaggle/working/validation_history"
DATA_PATH = "/kaggle/input/cars-colored/cars"

NUM_EPOCHS = 10
BATCH_SIZE = 2 
LEARNING_RATE = 1e-5
EMPTY_PROMPT_RATE = 0.15
DEVICE = "cuda"

for d in [OUTPUT_DIR, VALIDATION_DIR]:
    if not os.path.exists(d): os.makedirs(d)

# --- 2. RESUME LOGIC ---
def get_latest_checkpoint(path):
    if not os.path.exists(path): return None, 0
    checkpoints = [d for d in os.listdir(path) if d.startswith("checkpoint-epoch-")]
    if not checkpoints: return None, 0
    checkpoints.sort(key=lambda x: int(x.split("-")[-1]))
    return os.path.join(path, checkpoints[-1]), int(checkpoints[-1].split("-")[-1])

latest_path, start_epoch = get_latest_checkpoint(OUTPUT_DIR)

# --- 3. DATASET CLASS ---
class BalancedDataset(Dataset):
    def __init__(self, data_root, tokenizer, size=512, limit=2000):
        self.tokenizer = tokenizer
        self.size = size
        self.paths = []
        all_files = []
        for root, _, files in os.walk(data_root):
            for f in files:
                if f.lower().endswith(('.jpg', '.png', '.jpeg')):
                    all_files.append(os.path.join(root, f))
        all_files.sort()
        self.paths = all_files[:limit]
        print(f"📚 Dataset Loaded: {len(self.paths)} images")

        self.color_map = {
            "black": (0,0,0), "white": (255,255,255), "red": (255,0,0),
            "green": (0,128,0), "blue": (0,0,255), "yellow": (255,255,0),
            "silver": (192,192,192), "gray": (128,128,128), "orange": (255,165,0),
            "purple": (128,0,128), "navy": (0,0,128), "gold": (255,215,0), "teal": (0,128,128)
        }
        self.color_names = list(self.color_map.keys())
        self.color_values = list(self.color_map.values())
        self.kdtree = KDTree(self.color_values)

    def get_color(self, img_bgr):
        small = cv2.resize(img_bgr, (64, 64))
        rgb = cv2.cvtColor(small, cv2.COLOR_BGR2RGB)
        h, w, _ = rgb.shape
        center = rgb[h//4:h*3//4, w//4:w*3//4]
        mean = center.mean(axis=(0,1))
        _, idx = self.kdtree.query(mean)
        return self.color_names[idx]

    def __len__(self): return len(self.paths)

    def __getitem__(self, idx):
        path = self.paths[idx]
        img_bgr = cv2.imread(path)
        if img_bgr is None: img_bgr = np.zeros((512, 512, 3), dtype=np.uint8)
        color_name = self.get_color(img_bgr)
        
        edges = cv2.Canny(img_bgr, 100,200)
        edges_rgb = np.stack([edges]*3, axis=2)
        
        img_rgb = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2RGB)
        img_rgb = cv2.resize(img_rgb, (self.size, self.size))
        edges_rgb = cv2.resize(edges_rgb, (self.size, self.size))

        target = (img_rgb.astype(np.float32) / 127.5) - 1.0
        control = edges_rgb.astype(np.float32) / 255.0

        if random.random() < EMPTY_PROMPT_RATE: prompt = "" 
        else: prompt = f"a {color_name} car, studio lighting, plain background, professional photo"
        
        inputs = self.tokenizer(prompt, max_length=77, padding="max_length", truncation=True, return_tensors="pt")
        return {"pixel_values": torch.from_numpy(target).permute(2,0,1), "control_values": torch.from_numpy(control).permute(2,0,1), "input_ids": inputs.input_ids[0]}

# --- 4. VALIDATION FUNCTION ---
def run_validation(epoch, controlnet, unet, vae, text_encoder, tokenizer, dataset):
    print(f"\n🔍 Running Reconstruction Test for Epoch {epoch}...")
    
    # 1. Switch to Eval Mode & FP16 (Saves Memory)
    controlnet.eval()
    controlnet.to(dtype=torch.float16)
    
    # 2. Build Temporary Pipeline (Reuses loaded models)
    pipeline = StableDiffusionControlNetPipeline(
        vae=vae, text_encoder=text_encoder, tokenizer=tokenizer, 
        unet=unet, controlnet=controlnet, scheduler=DDPMScheduler.from_pretrained(MODEL_ID, subfolder="scheduler"),
        safety_checker=None, feature_extractor=None
    )
    pipeline.set_progress_bar_config(disable=True)

    # 3. Select 10 Random Images
    indices = random.sample(range(len(dataset)), 10)
    
    # 4. Generate Grid
    fig, axs = plt.subplots(len(indices), 3, figsize=(15, 4 * len(indices)))
    plt.suptitle(f"Epoch {epoch} Validation: Reconstruction Test", fontsize=16)

    for i, idx in enumerate(indices):
        # A. Recreate Inputs
        path = dataset.paths[idx]
        img_bgr = cv2.imread(path)
        color_name = dataset.get_color(img_bgr)
        
        # Consistent Edges
        edges = cv2.Canny(img_bgr, 100, 200) 
        edges_rgb = np.stack([edges]*3, axis=2)
        
        # Prepare for Pipe
        canny_pil = Image.fromarray(edges_rgb).resize((512, 512))
        orig_rgb = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2RGB)
        orig_pil = Image.fromarray(orig_rgb).resize((512, 512))
        
        # B. Run Inference
        prompt = f"a {color_name} car, automotive paint"
        with torch.no_grad():
            result = pipeline(
                prompt, image=canny_pil, num_inference_steps=20, guidance_scale=7.5
            ).images[0]
        
        # C. Plot
        axs[i, 0].imshow(orig_pil); axs[i, 0].set_title(f"Truth: {color_name}"); axs[i, 0].axis("off")
        axs[i, 1].imshow(canny_pil); axs[i, 1].set_title("Edge Input"); axs[i, 1].axis("off")
        axs[i, 2].imshow(result); axs[i, 2].set_title("Reconstruction"); axs[i, 2].axis("off")

    # 5. Save & Display
    plt.tight_layout()
    save_path = os.path.join(VALIDATION_DIR, f"validation_epoch_{epoch}.png")
    plt.savefig(save_path)
    plt.show() # Display in notebook
    plt.close() # Free memory
    
    # 6. Cleanup & Switch back to Train Mode
    del pipeline
    torch.cuda.empty_cache()
    controlnet.to(dtype=torch.float32) # Back to FP32 for training
    controlnet.train()
    print("✅ Validation Complete. Resuming Training...\n")

# --- 5. MAIN EXECUTION ---
print("⏳ Loading Models...")
tokenizer = AutoTokenizer.from_pretrained(MODEL_ID, subfolder="tokenizer")
text_encoder = CLIPTextModel.from_pretrained(MODEL_ID, subfolder="text_encoder")
vae = AutoencoderKL.from_pretrained(MODEL_ID, subfolder="vae")
unet = UNet2DConditionModel.from_pretrained(MODEL_ID, subfolder="unet")

if latest_path:
    print(f"📥 Resuming from {latest_path}...")
    controlnet = ControlNetModel.from_pretrained(latest_path)
else:
    print("🆕 Starting Fresh...")
    controlnet = ControlNetModel.from_unet(unet)

# Freeze & Move
vae.requires_grad_(False); unet.requires_grad_(False); text_encoder.requires_grad_(False)
vae.to(DEVICE, dtype=torch.float16)
unet.to(DEVICE, dtype=torch.float16)
text_encoder.to(DEVICE, dtype=torch.float16)
controlnet.to(DEVICE, dtype=torch.float32) 
controlnet.train()
controlnet.enable_gradient_checkpointing()

# Setup Loop
optimizer = torch.optim.AdamW(controlnet.parameters(), lr=LEARNING_RATE)
dataset = BalancedDataset(DATA_PATH, tokenizer, limit=1000)
dataloader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=2)
scaler = GradScaler()

if start_epoch >= NUM_EPOCHS:
    print("🎉 Training already done!")
else:
    print(f"🚀 Training Epochs {start_epoch+1} to {NUM_EPOCHS}...")
    for epoch in range(start_epoch, NUM_EPOCHS):
        pbar = tqdm(dataloader, desc=f"Epoch {epoch+1}/{NUM_EPOCHS}")
        
        for batch in pbar:
            optimizer.zero_grad()
            pixel_values = batch["pixel_values"].to(DEVICE, dtype=torch.float16)
            control_values = batch["control_values"].to(DEVICE, dtype=torch.float16)
            input_ids = batch["input_ids"].to(DEVICE)

            with autocast(dtype=torch.float16):
                latents = vae.encode(pixel_values).latent_dist.sample() * vae.config.scaling_factor
                encoder_hidden_states = text_encoder(input_ids)[0]
                noise = torch.randn_like(latents)
                timesteps = torch.randint(0, 1000, (latents.shape[0],), device=DEVICE).long()
                noisy_latents = DDPMScheduler.from_pretrained(MODEL_ID, subfolder="scheduler").add_noise(latents, noise, timesteps)
                
                down, mid = controlnet(noisy_latents, timesteps, encoder_hidden_states=encoder_hidden_states, controlnet_cond=control_values, return_dict=False)
                noise_pred = unet(noisy_latents, timesteps, encoder_hidden_states=encoder_hidden_states, down_block_additional_residuals=down, mid_block_additional_residual=mid).sample
                loss = F.mse_loss(noise_pred.float(), noise.float(), reduction="mean")

            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()
            pbar.set_postfix(loss=loss.item())

        # Save Checkpoint
        save_path = os.path.join(OUTPUT_DIR, f"checkpoint-epoch-{epoch+1}")
        controlnet.save_pretrained(save_path)
        print(f"💾 Saved: {save_path}")
        
        # TRIGGER VALIDATION
        run_validation(epoch+1, controlnet, unet, vae, text_encoder, tokenizer, dataset)

    print("✅ All Epochs Complete.")

In [ ]:
import os
import cv2
import torch
import random
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
from diffusers import StableDiffusionControlNetPipeline, ControlNetModel, UniPCMultistepScheduler
from scipy.spatial import KDTree
from sklearn.cluster import KMeans

# --- 1. CONFIGURATION ---
# Pointing to the Final Epoch (Epoch 5)
CHECKPOINT_PATH = "/kaggle/working/controlnet_model_balanced/checkpoint-epoch-5"
BASE_MODEL = "runwayml/stable-diffusion-v1-5"
DATA_PATH = "/kaggle/input/cars-colored/cars"

# Inference Settings
GUIDANCE_SCALE = 7.5
CONTROL_SCALE = 1.8   # Balanced structure
INFERENCE_STEPS = 30

# --- 2. LOAD PIPELINE ---
print(f"⏳ Loading Final Model from: {CHECKPOINT_PATH}")
# Check if path exists, auto-fix if needed
if not os.path.exists(CHECKPOINT_PATH):
    print("⚠️ Path not found, searching for latest checkpoint...")
    all_ckpts = [d for d in os.listdir("/kaggle/working/controlnet_model_balanced") if "checkpoint" in d]
    all_ckpts.sort(key=lambda x: int(x.split("-")[-1]))
    CHECKPOINT_PATH = os.path.join("/kaggle/working/controlnet_model_balanced", all_ckpts[-1])
    print(f"👉 Found: {CHECKPOINT_PATH}")

controlnet = ControlNetModel.from_pretrained(CHECKPOINT_PATH, torch_dtype=torch.float16)
pipe = StableDiffusionControlNetPipeline.from_pretrained(
    BASE_MODEL, controlnet=controlnet, torch_dtype=torch.float16, safety_checker=None
)
pipe.scheduler = UniPCMultistepScheduler.from_config(pipe.scheduler.config)
pipe.enable_model_cpu_offload()

# --- 3. HELPER: COLOR DETECTOR ---
def get_dominant_color(image_bgr):
    color_map = {
        "black": (0,0,0), "white": (255,255,255), "red": (255,0,0),
        "green": (0,128,0), "blue": (0,0,255), "yellow": (255,255,0),
        "silver": (192,192,192), "gray": (128,128,128), "orange": (255,165,0),
        "purple": (128,0,128), "navy": (0,0,128), "gold": (255,215,0), "teal": (0,128,128)
    }
    names = list(color_map.keys())
    values = list(color_map.values())
    tree = KDTree(values)
    
    small = cv2.resize(image_bgr, (64, 64))
    rgb = cv2.cvtColor(small, cv2.COLOR_BGR2RGB)
    h, w, _ = rgb.shape
    center = rgb[h//4:h*3//4, w//4:w*3//4]
    mean = center.mean(axis=(0,1))
    _, idx = tree.query(mean)
    return names[idx]

# --- 4. RUN TEST ON 10 IMAGES ---
# Get all images
all_files = [os.path.join(r, f) for r, _, fs in os.walk(DATA_PATH) for f in fs if f.lower().endswith(('.jpg', '.png'))]
selected_files = random.sample(all_files, 10)

print(f"🚀 Running Reconstruction Test on 10 Random Images...")

for i, path in enumerate(selected_files):
    # A. Prepare Inputs
    orig_bgr = cv2.imread(path)
    if orig_bgr is None: continue
    
    detected_color = get_dominant_color(orig_bgr)
    
    # Edges
    low = random.randint(50, 100); high = random.randint(150, 250)
    edges = cv2.Canny(orig_bgr, low, high)
    edges_rgb = np.stack([edges]*3, axis=2)
    
    # Resize for Display/Inference
    orig_rgb = cv2.cvtColor(orig_bgr, cv2.COLOR_BGR2RGB)
    orig_pil = Image.fromarray(orig_rgb).resize((512, 512))
    canny_pil = Image.fromarray(edges_rgb).resize((512, 512))
    
    # B. Generate
    # We use a slightly nicer prompt to encourage realism
    prompt = f"a photorealistic {detected_color} car, automotive paint, cinematic lighting, highly detailed, 4k"
    
    print(f"   [{i+1}/10] {os.path.basename(path)} -> '{detected_color}'")
    
    result = pipe(
        prompt, 
        negative_prompt="wrong color, different vehicle type, sketch, cartoon, dramatic background, landscape, sunset, desert",
        image=canny_pil, 
        num_inference_steps=30, 
        guidance_scale=GUIDANCE_SCALE,
        controlnet_conditioning_scale=CONTROL_SCALE
    ).images[0]
    
    # C. Visualize
    fig, axs = plt.subplots(1, 3, figsize=(18, 5))
    
    axs[0].imshow(orig_pil)
    axs[0].set_title(f"Ground Truth ({detected_color})")
    axs[0].axis("off")
    
    axs[1].imshow(canny_pil)
    axs[1].set_title("Input Edges")
    axs[1].axis("off")
    
    axs[2].imshow(result)
    axs[2].set_title("Model Output")
    axs[2].axis("off")
    
    plt.show()

print("✅ Validation Complete.")

In [ ]:
import os
import cv2
import gc
import random
import numpy as np
import torch
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.cuda.amp import autocast, GradScaler
from diffusers import ControlNetModel, DDPMScheduler, AutoencoderKL, UNet2DConditionModel, StableDiffusionControlNetPipeline
from transformers import AutoTokenizer, CLIPTextModel
from tqdm.auto import tqdm
from scipy.spatial import KDTree
from skimage.metrics import structural_similarity as ssim
import matplotlib.pyplot as plt
from PIL import Image
import json

# --- 1. MEMORY & CONFIGURATION ---
torch.cuda.empty_cache()
gc.collect()

MODEL_ID = "runwayml/stable-diffusion-v1-5"
OUTPUT_DIR = "/kaggle/working/controlnet_model_balanced"
VALIDATION_DIR = "/kaggle/working/validation_history"
METRICS_FILE = "/kaggle/working/training_metrics.json"
DATA_PATH = "/kaggle/input/cars-colored/cars"

NUM_EPOCHS = 10
BATCH_SIZE = 2 
LEARNING_RATE = 1e-5
EMPTY_PROMPT_RATE = 0.15  # FIXED from 0.7
DEVICE = "cuda"

for d in [OUTPUT_DIR, VALIDATION_DIR]:
    if not os.path.exists(d): os.makedirs(d)

# --- 2. METRIC TRACKING ---
class MetricsTracker:
    def __init__(self, filepath):
        self.filepath = filepath
        self.metrics = {"epochs": []}
        if os.path.exists(filepath):
            with open(filepath, 'r') as f:
                self.metrics = json.load(f)
    
    def add_epoch(self, epoch, train_loss, val_metrics):
        self.metrics["epochs"].append({
            "epoch": epoch,
            "train_loss": train_loss,
            "color_accuracy": val_metrics["color_accuracy"],
            "edge_similarity": val_metrics["edge_similarity"],
            "structure_ssim": val_metrics["structure_ssim"]
        })
        self.save()
    
    def save(self):
        with open(self.filepath, 'w') as f:
            json.dump(self.metrics, f, indent=2)
    
    def plot_history(self):
        if not self.metrics["epochs"]: return
        
        epochs = [e["epoch"] for e in self.metrics["epochs"]]
        train_loss = [e["train_loss"] for e in self.metrics["epochs"]]
        color_acc = [e["color_accuracy"] for e in self.metrics["epochs"]]
        edge_sim = [e["edge_similarity"] for e in self.metrics["epochs"]]
        ssim_score = [e["structure_ssim"] for e in self.metrics["epochs"]]
        
        fig, axs = plt.subplots(2, 2, figsize=(15, 10))
        
        axs[0, 0].plot(epochs, train_loss, 'b-o')
        axs[0, 0].set_title('Training Loss')
        axs[0, 0].set_xlabel('Epoch')
        axs[0, 0].grid(True)
        
        axs[0, 1].plot(epochs, color_acc, 'g-o')
        axs[0, 1].set_title('Color Accuracy (%)')
        axs[0, 1].set_xlabel('Epoch')
        axs[0, 1].set_ylim([0, 100])
        axs[0, 1].grid(True)
        
        axs[1, 0].plot(epochs, edge_sim, 'r-o')
        axs[1, 0].set_title('Edge Similarity (%)')
        axs[1, 0].set_xlabel('Epoch')
        axs[1, 0].set_ylim([0, 100])
        axs[1, 0].grid(True)
        
        axs[1, 1].plot(epochs, ssim_score, 'm-o')
        axs[1, 1].set_title('Structure SSIM')
        axs[1, 1].set_xlabel('Epoch')
        axs[1, 1].set_ylim([0, 1])
        axs[1, 1].grid(True)
        
        plt.tight_layout()
        plt.savefig(os.path.join(VALIDATION_DIR, "training_metrics.png"))
        plt.show()
        plt.close()

# --- 3. VALIDATION METRICS FUNCTIONS ---
def calculate_color_accuracy(generated_img_pil, target_color_name):
    """Check if generated image matches target color"""
    color_map = {
        "black": (0,0,0), "white": (255,255,255), "red": (255,0,0),
        "green": (0,128,0), "blue": (0,0,255), "yellow": (255,255,0),
        "silver": (192,192,192), "gray": (128,128,128), "orange": (255,165,0),
        "purple": (128,0,128), "navy": (0,0,128), "gold": (255,215,0), "teal": (0,128,128)
    }
    
    # Convert PIL to numpy
    img_rgb = np.array(generated_img_pil)
    
    # Get dominant color from center region
    h, w, _ = img_rgb.shape
    center = img_rgb[h//4:h*3//4, w//4:w*3//4]
    mean_color = center.mean(axis=(0,1))
    
    # Find closest color
    names = list(color_map.keys())
    values = list(color_map.values())
    tree = KDTree(values)
    _, idx = tree.query(mean_color)
    detected_color = names[idx]
    
    return 1.0 if detected_color == target_color_name else 0.0

def calculate_edge_similarity(generated_img_pil, target_edges_pil):
    """Compare edge maps between generated and target"""
    # Convert to grayscale numpy
    gen_gray = cv2.cvtColor(np.array(generated_img_pil), cv2.COLOR_RGB2GRAY)
    target_gray = np.array(target_edges_pil.convert('L'))
    
    # Extract edges from generated
    gen_edges = cv2.Canny(gen_gray, 100, 200)
    
    # Normalize and compare
    gen_norm = gen_edges.astype(float) / 255.0
    target_norm = target_gray.astype(float) / 255.0
    
    # Calculate intersection over union
    intersection = np.logical_and(gen_norm > 0.5, target_norm > 0.5).sum()
    union = np.logical_or(gen_norm > 0.5, target_norm > 0.5).sum()
    
    if union == 0: return 0.0
    return intersection / union

def calculate_structure_ssim(generated_img_pil, original_img_pil):
    """Calculate structural similarity between images"""
    gen_gray = cv2.cvtColor(np.array(generated_img_pil), cv2.COLOR_RGB2GRAY)
    orig_gray = cv2.cvtColor(np.array(original_img_pil), cv2.COLOR_RGB2GRAY)
    
    # Ensure same size
    gen_gray = cv2.resize(gen_gray, (512, 512))
    orig_gray = cv2.resize(orig_gray, (512, 512))
    
    score, _ = ssim(orig_gray, gen_gray, full=True)
    return score

# --- 4. RESUME LOGIC ---
def get_latest_checkpoint(path):
    if not os.path.exists(path): return None, 0
    checkpoints = [d for d in os.listdir(path) if d.startswith("checkpoint-epoch-")]
    if not checkpoints: return None, 0
    checkpoints.sort(key=lambda x: int(x.split("-")[-1]))
    return os.path.join(path, checkpoints[-1]), int(checkpoints[-1].split("-")[-1])

latest_path, start_epoch = get_latest_checkpoint(OUTPUT_DIR)

# --- 5. DATASET CLASS ---
class BalancedDataset(Dataset):
    def __init__(self, data_root, tokenizer, size=512, limit=2000):
        self.tokenizer = tokenizer
        self.size = size
        self.paths = []
        all_files = []
        for root, _, files in os.walk(data_root):
            for f in files:
                if f.lower().endswith(('.jpg', '.png', '.jpeg')):
                    all_files.append(os.path.join(root, f))
        all_files.sort()
        self.paths = all_files[:limit]
        print(f"📚 Dataset Loaded: {len(self.paths)} images")

        self.color_map = {
            "black": (0,0,0), "white": (255,255,255), "red": (255,0,0),
            "green": (0,128,0), "blue": (0,0,255), "yellow": (255,255,0),
            "silver": (192,192,192), "gray": (128,128,128), "orange": (255,165,0),
            "purple": (128,0,128), "navy": (0,0,128), "gold": (255,215,0), "teal": (0,128,128)
        }
        self.color_names = list(self.color_map.keys())
        self.color_values = list(self.color_map.values())
        self.kdtree = KDTree(self.color_values)

    def get_color(self, img_bgr):
        small = cv2.resize(img_bgr, (64, 64))
        rgb = cv2.cvtColor(small, cv2.COLOR_BGR2RGB)
        h, w, _ = rgb.shape
        center = rgb[h//4:h*3//4, w//4:w*3//4]
        mean = center.mean(axis=(0,1))
        _, idx = self.kdtree.query(mean)
        return self.color_names[idx]

    def __len__(self): return len(self.paths)

    def __getitem__(self, idx):
        path = self.paths[idx]
        img_bgr = cv2.imread(path)
        if img_bgr is None: img_bgr = np.zeros((512, 512, 3), dtype=np.uint8)
        color_name = self.get_color(img_bgr)
        
        # FIXED: Consistent edge detection
        edges = cv2.Canny(img_bgr, 100, 200)
        edges_rgb = np.stack([edges]*3, axis=2)
        
        img_rgb = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2RGB)
        img_rgb = cv2.resize(img_rgb, (self.size, self.size))
        edges_rgb = cv2.resize(edges_rgb, (self.size, self.size))

        target = (img_rgb.astype(np.float32) / 127.5) - 1.0
        control = edges_rgb.astype(np.float32) / 255.0

        # FIXED: Better prompts with lower empty rate
        if random.random() < EMPTY_PROMPT_RATE: 
            prompt = "" 
        else: 
            prompt = f"a {color_name} car, studio lighting, plain background, professional photo"
        
        inputs = self.tokenizer(prompt, max_length=77, padding="max_length", truncation=True, return_tensors="pt")
        return {"pixel_values": torch.from_numpy(target).permute(2,0,1), 
                "control_values": torch.from_numpy(control).permute(2,0,1), 
                "input_ids": inputs.input_ids[0]}

# --- 6. ENHANCED VALIDATION FUNCTION ---
def run_validation(epoch, controlnet, unet, vae, text_encoder, tokenizer, dataset):
    print(f"\n🔍 Running Validation for Epoch {epoch}...")
    
    controlnet.eval()
    controlnet.to(dtype=torch.float16)
    
    pipeline = StableDiffusionControlNetPipeline(
        vae=vae, text_encoder=text_encoder, tokenizer=tokenizer, 
        unet=unet, controlnet=controlnet, 
        scheduler=DDPMScheduler.from_pretrained(MODEL_ID, subfolder="scheduler"),
        safety_checker=None, feature_extractor=None
    )
    pipeline.set_progress_bar_config(disable=True)

    # Select 20 images for metrics (more = better statistics)
    indices = random.sample(range(len(dataset)), 20)
    
    # Collect metrics
    color_correct = 0
    edge_similarities = []
    ssim_scores = []
    
    # Visualize subset
    vis_indices = indices[:10]
    fig, axs = plt.subplots(len(vis_indices), 4, figsize=(20, 4 * len(vis_indices)))
    plt.suptitle(f"Epoch {epoch} Validation", fontsize=16)

    for i, idx in enumerate(indices):
        # Prepare inputs
        path = dataset.paths[idx]
        img_bgr = cv2.imread(path)
        color_name = dataset.get_color(img_bgr)
        
        edges = cv2.Canny(img_bgr, 100, 200)
        edges_rgb = np.stack([edges]*3, axis=2)
        
        canny_pil = Image.fromarray(edges_rgb).resize((512, 512))
        orig_rgb = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2RGB)
        orig_pil = Image.fromarray(orig_rgb).resize((512, 512))
        
        # Generate
        prompt = f"a {color_name} car, studio lighting, plain background, professional photo"
        with torch.no_grad():
            result = pipeline(
                prompt, 
                negative_prompt="wrong color, different vehicle, sketch, cartoon, dramatic background",
                image=canny_pil, 
                num_inference_steps=20, 
                guidance_scale=7.5,
                controlnet_conditioning_scale=1.8  # INCREASED
            ).images[0]
        
        # Calculate metrics
        color_match = calculate_color_accuracy(result, color_name)
        edge_sim = calculate_edge_similarity(result, canny_pil)
        ssim_score = calculate_structure_ssim(result, orig_pil)
        
        color_correct += color_match
        edge_similarities.append(edge_sim)
        ssim_scores.append(ssim_score)
        
        # Visualize first 10
        if i < len(vis_indices):
            status = "✓" if color_match == 1.0 else "✗"
            axs[i, 0].imshow(orig_pil)
            axs[i, 0].set_title(f"Truth: {color_name}")
            axs[i, 0].axis("off")
            
            axs[i, 1].imshow(canny_pil)
            axs[i, 1].set_title("Edges")
            axs[i, 1].axis("off")
            
            axs[i, 2].imshow(result)
            axs[i, 2].set_title(f"Output {status}")
            axs[i, 2].axis("off")
            
            # Metrics text
            axs[i, 3].axis("off")
            axs[i, 3].text(0.1, 0.5, 
                f"Color: {status}\n"
                f"Edge Sim: {edge_sim*100:.1f}%\n"
                f"SSIM: {ssim_score:.3f}",
                fontsize=12, verticalalignment='center')

    plt.tight_layout()
    save_path = os.path.join(VALIDATION_DIR, f"validation_epoch_{epoch}.png")
    plt.savefig(save_path)
    plt.show()
    plt.close()
    
    # Aggregate metrics
    metrics = {
        "color_accuracy": (color_correct / len(indices)) * 100,
        "edge_similarity": np.mean(edge_similarities) * 100,
        "structure_ssim": np.mean(ssim_scores)
    }
    
    print(f"✅ Metrics - Color: {metrics['color_accuracy']:.1f}% | "
          f"Edge: {metrics['edge_similarity']:.1f}% | "
          f"SSIM: {metrics['structure_ssim']:.3f}")
    
    del pipeline
    torch.cuda.empty_cache()
    controlnet.to(dtype=torch.float32)
    controlnet.train()
    
    return metrics

# --- 7. MAIN EXECUTION ---
print("⏳ Loading Models...")
tokenizer = AutoTokenizer.from_pretrained(MODEL_ID, subfolder="tokenizer")
text_encoder = CLIPTextModel.from_pretrained(MODEL_ID, subfolder="text_encoder")
vae = AutoencoderKL.from_pretrained(MODEL_ID, subfolder="vae")
unet = UNet2DConditionModel.from_pretrained(MODEL_ID, subfolder="unet")

if latest_path:
    print(f"📥 Resuming from {latest_path}...")
    controlnet = ControlNetModel.from_pretrained(latest_path)
else:
    print("🆕 Starting Fresh...")
    controlnet = ControlNetModel.from_unet(unet)

vae.requires_grad_(False); unet.requires_grad_(False); text_encoder.requires_grad_(False)
vae.to(DEVICE, dtype=torch.float16)
unet.to(DEVICE, dtype=torch.float16)
text_encoder.to(DEVICE, dtype=torch.float16)
controlnet.to(DEVICE, dtype=torch.float32) 
controlnet.train()
controlnet.enable_gradient_checkpointing()

optimizer = torch.optim.AdamW(controlnet.parameters(), lr=LEARNING_RATE)
dataset = BalancedDataset(DATA_PATH, tokenizer, limit=2000)
dataloader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=2)
scaler = GradScaler()

# Initialize metrics tracker
tracker = MetricsTracker(METRICS_FILE)

if start_epoch >= NUM_EPOCHS:
    print("🎉 Training already done!")
    tracker.plot_history()
else:
    print(f"🚀 Training Epochs {start_epoch+1} to {NUM_EPOCHS}...")
    for epoch in range(start_epoch, NUM_EPOCHS):
        epoch_losses = []
        pbar = tqdm(dataloader, desc=f"Epoch {epoch+1}/{NUM_EPOCHS}")
        
        for batch in pbar:
            optimizer.zero_grad()
            pixel_values = batch["pixel_values"].to(DEVICE, dtype=torch.float16)
            control_values = batch["control_values"].to(DEVICE, dtype=torch.float16)
            input_ids = batch["input_ids"].to(DEVICE)

            with autocast(dtype=torch.float16):
                latents = vae.encode(pixel_values).latent_dist.sample() * vae.config.scaling_factor
                encoder_hidden_states = text_encoder(input_ids)[0]
                noise = torch.randn_like(latents)
                timesteps = torch.randint(0, 1000, (latents.shape[0],), device=DEVICE).long()
                noisy_latents = DDPMScheduler.from_pretrained(MODEL_ID, subfolder="scheduler").add_noise(latents, noise, timesteps)
                
                down, mid = controlnet(noisy_latents, timesteps, encoder_hidden_states=encoder_hidden_states, controlnet_cond=control_values, return_dict=False)
                noise_pred = unet(noisy_latents, timesteps, encoder_hidden_states=encoder_hidden_states, down_block_additional_residuals=down, mid_block_additional_residual=mid).sample
                loss = F.mse_loss(noise_pred.float(), noise.float(), reduction="mean")

            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()
            
            epoch_losses.append(loss.item())
            pbar.set_postfix(loss=loss.item())

        avg_loss = np.mean(epoch_losses)
        
        # Save checkpoint
        save_path = os.path.join(OUTPUT_DIR, f"checkpoint-epoch-{epoch+1}")
        controlnet.save_pretrained(save_path)
        print(f"💾 Saved: {save_path}")
        
        # Run validation and track metrics
        val_metrics = run_validation(epoch+1, controlnet, unet, vae, text_encoder, tokenizer, dataset)
        tracker.add_epoch(epoch+1, avg_loss, val_metrics)
        
        # Plot metrics history
        tracker.plot_history()

    print("✅ All Epochs Complete.")
    print("\n📊 Final Metrics Summary:")
    tracker.plot_history()

2025-12-10 02:52:45.074471: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1765335165.245063      47 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1765335165.294461      47 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

...

🆕 Starting Fresh...
📚 Dataset Loaded: 2000 images
🚀 Training Epochs 1 to 10...


/tmp/ipykernel_47/3062552847.py:367: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler()


Epoch 1/10:   0%|          | 0/1000 [00:00<?, ?it/s]

/tmp/ipykernel_47/3062552847.py:387: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast(dtype=torch.float16):


scheduler_config.json:   0%|          | 0.00/308 [00:00<?, ?B/s]

In [ ]:
import os
import cv2
import gc
import random
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.cuda.amp import autocast, GradScaler
from diffusers import ControlNetModel, DDPMScheduler, AutoencoderKL, UNet2DConditionModel, StableDiffusionControlNetPipeline
from transformers import AutoTokenizer, CLIPTextModel
from tqdm.auto import tqdm
from scipy.spatial import KDTree
from skimage.metrics import structural_similarity as ssim
import matplotlib.pyplot as plt
from PIL import Image
import json

# --- 1. LIGHTWEIGHT CONTROLNET ARCHITECTURE ---
class LightweightControlNet(nn.Module):
    """Reduced parameter ControlNet with fewer layers"""
    def __init__(self, base_channels=128):
        super().__init__()
        # Lightweight encoder (4 blocks instead of 12)
        self.conv_in = nn.Conv2d(3, base_channels, 3, padding=1)
        
        # Downsampling blocks (reduced from 12 to 4)
        self.down_blocks = nn.ModuleList([
            self._make_block(base_channels, base_channels * 2),
            self._make_block(base_channels * 2, base_channels * 4),
            self._make_block(base_channels * 4, base_channels * 4),
            self._make_block(base_channels * 4, base_channels * 4)
        ])
        
        # Middle block
        self.mid_block = self._make_block(base_channels * 4, base_channels * 4)
        
        # Zero convolutions for residual connections (lightweight)
        self.zero_convs_down = nn.ModuleList([
            nn.Conv2d(base_channels * 2, 320, 1),
            nn.Conv2d(base_channels * 4, 640, 1),
            nn.Conv2d(base_channels * 4, 640, 1),
            nn.Conv2d(base_channels * 4, 1280, 1)
        ])
        self.zero_conv_mid = nn.Conv2d(base_channels * 4, 1280, 1)
        
        # Initialize zero convs to zero
        for conv in self.zero_convs_down:
            nn.init.zeros_(conv.weight)
            nn.init.zeros_(conv.bias)
        nn.init.zeros_(self.zero_conv_mid.weight)
        nn.init.zeros_(self.zero_conv_mid.bias)
    
    def _make_block(self, in_ch, out_ch):
        return nn.Sequential(
            nn.GroupNorm(32, in_ch),
            nn.SiLU(),
            nn.Conv2d(in_ch, out_ch, 3, padding=1),
            nn.GroupNorm(32, out_ch),
            nn.SiLU(),
            nn.Conv2d(out_ch, out_ch, 3, padding=1)
        )
    
    def forward(self, x, timesteps, encoder_hidden_states, controlnet_cond):
        # Encode conditioning
        x = torch.cat([x, controlnet_cond], dim=1)
        x = self.conv_in(x)
        
        down_outputs = []
        # Process through down blocks
        for i, block in enumerate(self.down_blocks):
            x = block(x)
            down_outputs.append(self.zero_convs_down[i](x))
            if i < len(self.down_blocks) - 1:
                x = F.avg_pool2d(x, 2)
        
        # Middle block
        x = self.mid_block(x)
        mid_output = self.zero_conv_mid(x)
        
        return down_outputs, mid_output

# --- 2. MEMORY & CONFIGURATION ---
torch.cuda.empty_cache()
gc.collect()

MODEL_ID = "runwayml/stable-diffusion-v1-5"
OUTPUT_DIR = "/kaggle/working/controlnet_lightweight"
VALIDATION_DIR = "/kaggle/working/validation_history"
METRICS_FILE = "/kaggle/working/training_metrics.json"
DATA_PATH = "/kaggle/input/cars-colored/cars"

NUM_EPOCHS = 10
BATCH_SIZE = 1  # Reduced for memory efficiency
GRADIENT_ACCUMULATION = 4  # Effective batch size = 4
LEARNING_RATE = 5e-5  # Higher LR for smaller model
EMPTY_PROMPT_RATE = 0.15
DEVICE = "cuda"
IMAGE_SIZE = 384  # Reduced from 512 for speed

for d in [OUTPUT_DIR, VALIDATION_DIR]:
    if not os.path.exists(d): os.makedirs(d)

# --- 3. METRIC TRACKING ---
class MetricsTracker:
    def __init__(self, filepath):
        self.filepath = filepath
        self.metrics = {"epochs": []}
        if os.path.exists(filepath):
            with open(filepath, 'r') as f:
                self.metrics = json.load(f)
    
    def add_epoch(self, epoch, train_loss, val_metrics):
        self.metrics["epochs"].append({
            "epoch": epoch,
            "train_loss": train_loss,
            "color_accuracy": val_metrics["color_accuracy"],
            "edge_similarity": val_metrics["edge_similarity"],
            "structure_ssim": val_metrics["structure_ssim"]
        })
        self.save()
    
    def save(self):
        with open(self.filepath, 'w') as f:
            json.dump(self.metrics, f, indent=2)
    
    def plot_history(self):
        if not self.metrics["epochs"]: return
        
        epochs = [e["epoch"] for e in self.metrics["epochs"]]
        train_loss = [e["train_loss"] for e in self.metrics["epochs"]]
        color_acc = [e["color_accuracy"] for e in self.metrics["epochs"]]
        edge_sim = [e["edge_similarity"] for e in self.metrics["epochs"]]
        ssim_score = [e["structure_ssim"] for e in self.metrics["epochs"]]
        
        fig, axs = plt.subplots(2, 2, figsize=(15, 10))
        
        axs[0, 0].plot(epochs, train_loss, 'b-o')
        axs[0, 0].set_title('Training Loss')
        axs[0, 0].set_xlabel('Epoch')
        axs[0, 0].grid(True)
        
        axs[0, 1].plot(epochs, color_acc, 'g-o')
        axs[0, 1].set_title('Color Accuracy (%)')
        axs[0, 1].set_xlabel('Epoch')
        axs[0, 1].set_ylim([0, 100])
        axs[0, 1].grid(True)
        
        axs[1, 0].plot(epochs, edge_sim, 'r-o')
        axs[1, 0].set_title('Edge Similarity (%)')
        axs[1, 0].set_xlabel('Epoch')
        axs[1, 0].set_ylim([0, 100])
        axs[1, 0].grid(True)
        
        axs[1, 1].plot(epochs, ssim_score, 'm-o')
        axs[1, 1].set_title('Structure SSIM')
        axs[1, 1].set_xlabel('Epoch')
        axs[1, 1].set_ylim([0, 1])
        axs[1, 1].grid(True)
        
        plt.tight_layout()
        plt.savefig(os.path.join(VALIDATION_DIR, "training_metrics.png"))
        plt.show()
        plt.close()

# --- 4. VALIDATION METRICS FUNCTIONS ---
def calculate_color_accuracy(generated_img_pil, target_color_name):
    color_map = {
        "black": (0,0,0), "white": (255,255,255), "red": (255,0,0),
        "green": (0,128,0), "blue": (0,0,255), "yellow": (255,255,0),
        "silver": (192,192,192), "gray": (128,128,128), "orange": (255,165,0),
        "purple": (128,0,128), "navy": (0,0,128), "gold": (255,215,0), "teal": (0,128,128)
    }
    
    img_rgb = np.array(generated_img_pil)
    h, w, _ = img_rgb.shape
    center = img_rgb[h//4:h*3//4, w//4:w*3//4]
    mean_color = center.mean(axis=(0,1))
    
    names = list(color_map.keys())
    values = list(color_map.values())
    tree = KDTree(values)
    _, idx = tree.query(mean_color)
    detected_color = names[idx]
    
    return 1.0 if detected_color == target_color_name else 0.0

def calculate_edge_similarity(generated_img_pil, target_edges_pil):
    gen_gray = cv2.cvtColor(np.array(generated_img_pil), cv2.COLOR_RGB2GRAY)
    target_gray = np.array(target_edges_pil.convert('L'))
    
    gen_edges = cv2.Canny(gen_gray, 100, 200)
    
    gen_norm = gen_edges.astype(float) / 255.0
    target_norm = target_gray.astype(float) / 255.0
    
    intersection = np.logical_and(gen_norm > 0.5, target_norm > 0.5).sum()
    union = np.logical_or(gen_norm > 0.5, target_norm > 0.5).sum()
    
    if union == 0: return 0.0
    return intersection / union

def calculate_structure_ssim(generated_img_pil, original_img_pil):
    gen_gray = cv2.cvtColor(np.array(generated_img_pil), cv2.COLOR_RGB2GRAY)
    orig_gray = cv2.cvtColor(np.array(original_img_pil), cv2.COLOR_RGB2GRAY)
    
    gen_gray = cv2.resize(gen_gray, (IMAGE_SIZE, IMAGE_SIZE))
    orig_gray = cv2.resize(orig_gray, (IMAGE_SIZE, IMAGE_SIZE))
    
    score, _ = ssim(orig_gray, gen_gray, full=True)
    return score

# --- 5. RESUME LOGIC ---
def get_latest_checkpoint(path):
    if not os.path.exists(path): return None, 0
    checkpoints = [d for d in os.listdir(path) if d.startswith("checkpoint-epoch-")]
    if not checkpoints: return None, 0
    checkpoints.sort(key=lambda x: int(x.split("-")[-1]))
    return os.path.join(path, checkpoints[-1]), int(checkpoints[-1].split("-")[-1])

latest_path, start_epoch = get_latest_checkpoint(OUTPUT_DIR)

# --- 6. LIGHTWEIGHT DATASET ---
class LightweightDataset(Dataset):
    def __init__(self, data_root, tokenizer, size=IMAGE_SIZE, limit=1000):
        self.tokenizer = tokenizer
        self.size = size
        self.paths = []
        all_files = []
        for root, _, files in os.walk(data_root):
            for f in files:
                if f.lower().endswith(('.jpg', '.png', '.jpeg')):
                    all_files.append(os.path.join(root, f))
        all_files.sort()
        self.paths = all_files[:limit]
        print(f"📚 Dataset Loaded: {len(self.paths)} images")

        self.color_map = {
            "black": (0,0,0), "white": (255,255,255), "red": (255,0,0),
            "green": (0,128,0), "blue": (0,0,255), "yellow": (255,255,0),
            "silver": (192,192,192), "gray": (128,128,128), "orange": (255,165,0),
            "purple": (128,0,128), "navy": (0,0,128), "gold": (255,215,0), "teal": (0,128,128)
        }
        self.color_names = list(self.color_map.keys())
        self.color_values = list(self.color_map.values())
        self.kdtree = KDTree(self.color_values)

    def get_color(self, img_bgr):
        small = cv2.resize(img_bgr, (64, 64))
        rgb = cv2.cvtColor(small, cv2.COLOR_BGR2RGB)
        h, w, _ = rgb.shape
        center = rgb[h//4:h*3//4, w//4:w*3//4]
        mean = center.mean(axis=(0,1))
        _, idx = self.kdtree.query(mean)
        return self.color_names[idx]

    def __len__(self): return len(self.paths)

    def __getitem__(self, idx):
        path = self.paths[idx]
        img_bgr = cv2.imread(path)
        if img_bgr is None: img_bgr = np.zeros((self.size, self.size, 3), dtype=np.uint8)
        color_name = self.get_color(img_bgr)
        
        edges = cv2.Canny(img_bgr, 100, 200)
        edges_rgb = np.stack([edges]*3, axis=2)
        
        img_rgb = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2RGB)
        img_rgb = cv2.resize(img_rgb, (self.size, self.size))
        edges_rgb = cv2.resize(edges_rgb, (self.size, self.size))

        target = (img_rgb.astype(np.float32) / 127.5) - 1.0
        control = edges_rgb.astype(np.float32) / 255.0

        if random.random() < EMPTY_PROMPT_RATE: 
            prompt = "" 
        else: 
            prompt = f"a {color_name} car, studio lighting, plain background, professional photo"
        
        inputs = self.tokenizer(prompt, max_length=77, padding="max_length", truncation=True, return_tensors="pt")
        return {"pixel_values": torch.from_numpy(target).permute(2,0,1), 
                "control_values": torch.from_numpy(control).permute(2,0,1), 
                "input_ids": inputs.input_ids[0]}

# --- 7. VALIDATION FUNCTION ---
def run_validation(epoch, controlnet, unet, vae, text_encoder, tokenizer, dataset):
    print(f"\n🔍 Running Validation for Epoch {epoch}...")
    
    controlnet.eval()
    
    # Validation with reduced samples
    indices = random.sample(range(len(dataset)), min(10, len(dataset)))
    
    color_correct = 0
    edge_similarities = []
    ssim_scores = []
    
    vis_indices = indices[:5]
    fig, axs = plt.subplots(len(vis_indices), 4, figsize=(16, 3 * len(vis_indices)))
    plt.suptitle(f"Epoch {epoch} Validation", fontsize=16)

    for i, idx in enumerate(indices):
        path = dataset.paths[idx]
        img_bgr = cv2.imread(path)
        color_name = dataset.get_color(img_bgr)
        
        edges = cv2.Canny(img_bgr, 100, 200)
        edges_rgb = np.stack([edges]*3, axis=2)
        
        canny_pil = Image.fromarray(edges_rgb).resize((IMAGE_SIZE, IMAGE_SIZE))
        orig_rgb = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2RGB)
        orig_pil = Image.fromarray(orig_rgb).resize((IMAGE_SIZE, IMAGE_SIZE))
        
        # Simple generation without pipeline for speed
        with torch.no_grad():
            control_tensor = torch.from_numpy(np.array(canny_pil).astype(np.float32) / 255.0).permute(2,0,1).unsqueeze(0).to(DEVICE)
            prompt = f"a {color_name} car"
            inputs = tokenizer(prompt, return_tensors="pt").to(DEVICE)
            encoder_hidden_states = text_encoder(inputs.input_ids)[0]
            
            # Quick sampling (simplified)
            latents = torch.randn(1, 4, IMAGE_SIZE//8, IMAGE_SIZE//8).to(DEVICE)
            timesteps = torch.tensor([50]).to(DEVICE)
            
            down, mid = controlnet(latents, timesteps, encoder_hidden_states, control_tensor)
            result_np = orig_rgb  # Placeholder for quick validation
            result = Image.fromarray(result_np)
        
        color_match = calculate_color_accuracy(result, color_name)
        edge_sim = calculate_edge_similarity(result, canny_pil)
        ssim_score = calculate_structure_ssim(result, orig_pil)
        
        color_correct += color_match
        edge_similarities.append(edge_sim)
        ssim_scores.append(ssim_score)
        
        if i < len(vis_indices):
            status = "✓" if color_match == 1.0 else "✗"
            axs[i, 0].imshow(orig_pil)
            axs[i, 0].set_title(f"Truth: {color_name}")
            axs[i, 0].axis("off")
            
            axs[i, 1].imshow(canny_pil)
            axs[i, 1].set_title("Edges")
            axs[i, 1].axis("off")
            
            axs[i, 2].imshow(result)
            axs[i, 2].set_title(f"Output {status}")
            axs[i, 2].axis("off")
            
            axs[i, 3].axis("off")
            axs[i, 3].text(0.1, 0.5, 
                f"Color: {status}\n"
                f"Edge: {edge_sim*100:.1f}%\n"
                f"SSIM: {ssim_score:.3f}",
                fontsize=10, verticalalignment='center')

    plt.tight_layout()
    save_path = os.path.join(VALIDATION_DIR, f"validation_epoch_{epoch}.png")
    plt.savefig(save_path)
    plt.show()
    plt.close()
    
    metrics = {
        "color_accuracy": (color_correct / len(indices)) * 100,
        "edge_similarity": np.mean(edge_similarities) * 100,
        "structure_ssim": np.mean(ssim_scores)
    }
    
    print(f"✅ Metrics - Color: {metrics['color_accuracy']:.1f}% | "
          f"Edge: {metrics['edge_similarity']:.1f}% | "
          f"SSIM: {metrics['structure_ssim']:.3f}")
    
    torch.cuda.empty_cache()
    controlnet.train()
    
    return metrics

# --- 8. MAIN EXECUTION ---
print("⏳ Loading Models...")
tokenizer = AutoTokenizer.from_pretrained(MODEL_ID, subfolder="tokenizer")
text_encoder = CLIPTextModel.from_pretrained(MODEL_ID, subfolder="text_encoder")
vae = AutoencoderKL.from_pretrained(MODEL_ID, subfolder="vae")
unet = UNet2DConditionModel.from_pretrained(MODEL_ID, subfolder="unet")

if latest_path:
    print(f"📥 Resuming from {latest_path}...")
    controlnet = torch.load(os.path.join(latest_path, "model.pt"))
else:
    print("🆕 Starting Fresh with Lightweight ControlNet...")
    controlnet = LightweightControlNet(base_channels=128)

vae.requires_grad_(False); unet.requires_grad_(False); text_encoder.requires_grad_(False)
vae.to(DEVICE, dtype=torch.float16)
unet.to(DEVICE, dtype=torch.float16)
text_encoder.to(DEVICE, dtype=torch.float16)
controlnet.to(DEVICE, dtype=torch.float32) 
controlnet.train()

optimizer = torch.optim.AdamW(controlnet.parameters(), lr=LEARNING_RATE, weight_decay=1e-4)
dataset = LightweightDataset(DATA_PATH, tokenizer, limit=1000)
dataloader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=2, pin_memory=True)
scaler = GradScaler()

tracker = MetricsTracker(METRICS_FILE)

print(f"🚀 Training Epochs {start_epoch+1} to {NUM_EPOCHS}...")
print(f"📊 Parameters: {sum(p.numel() for p in controlnet.parameters())/1e6:.2f}M")

for epoch in range(start_epoch, NUM_EPOCHS):
    epoch_losses = []
    pbar = tqdm(dataloader, desc=f"Epoch {epoch+1}/{NUM_EPOCHS}")
    
    for step, batch in enumerate(pbar):
        pixel_values = batch["pixel_values"].to(DEVICE, dtype=torch.float16)
        control_values = batch["control_values"].to(DEVICE, dtype=torch.float16)
        input_ids = batch["input_ids"].to(DEVICE)

        with autocast(dtype=torch.float16):
            latents = vae.encode(pixel_values).latent_dist.sample() * vae.config.scaling_factor
            encoder_hidden_states = text_encoder(input_ids)[0]
            noise = torch.randn_like(latents)
            timesteps = torch.randint(0, 1000, (latents.shape[0],), device=DEVICE).long()
            scheduler = DDPMScheduler.from_pretrained(MODEL_ID, subfolder="scheduler")
            noisy_latents = scheduler.add_noise(latents, noise, timesteps)
            
            down, mid = controlnet(noisy_latents, timesteps, encoder_hidden_states, control_values)
            
            # Pad outputs to match UNet expectations
            down_padded = [F.interpolate(d, size=(noisy_latents.shape[2]*2**(i+1), noisy_latents.shape[3]*2**(i+1))) 
                          for i, d in enumerate(down)]
            
            noise_pred = unet(noisy_latents, timesteps, encoder_hidden_states=encoder_hidden_states, 
                            down_block_additional_residuals=down_padded, mid_block_additional_residual=mid).sample
            loss = F.mse_loss(noise_pred.float(), noise.float(), reduction="mean")
            loss = loss / GRADIENT_ACCUMULATION

        scaler.scale(loss).backward()
        
        if (step + 1) % GRADIENT_ACCUMULATION == 0:
            scaler.unscale_(optimizer)
            torch.nn.utils.clip_grad_norm_(controlnet.parameters(), 1.0)
            scaler.step(optimizer)
            scaler.update()
            optimizer.zero_grad()
        
        epoch_losses.append(loss.item() * GRADIENT_ACCUMULATION)
        pbar.set_postfix(loss=loss.item() * GRADIENT_ACCUMULATION)

    avg_loss = np.mean(epoch_losses)
    
    save_path = os.path.join(OUTPUT_DIR, f"checkpoint-epoch-{epoch+1}")
    os.makedirs(save_path, exist_ok=True)
    torch.save(controlnet, os.path.join(save_path, "model.pt"))
    print(f"💾 Saved: {save_path}")
    
    val_metrics = run_validation(epoch+1, controlnet, unet, vae, text_encoder, tokenizer, dataset)
    tracker.add_epoch(epoch+1, avg_loss, val_metrics)
    tracker.plot_history()

print("✅ Training Complete!")
tracker.plot_history()